# Tails
### PHYMOL Interactive Workshop, Cambridge, September 2024
### Konrad Patkowski, Auburn University (patkowsk@auburn.edu)

In [ ]:
"""Tails: Short-Range and Long-Range Behavior of Stuff"""

__authors__ = "Konrad Patkowski"
__email__   = ["patkowsk@auburn.edu"]

__copyright__ = "(c) 2024, Konrad Patkowski"
__license__   = "BSD-3-Clause"
__date__      = "2024-08-31"

# Why tails?

It is quite a common problem in computational chemistry to find a simplified but sensible representation of some quantity. This could be, for example, 
- a basis set representation for molecular orbitals, 
- a model potential representing some physical interaction in a force field,
- an analytical function approximating the potential energy surface for a molecule or an intermolecular complex.

As we develop such simplified representations, we care about *reproducing accurately the reference data, for example those from high-level calculations*. However, we also *don't want to mess up the performance of our representation away from the reference data, including its limiting behavior for very small or very large values of the distance/energy/... scale.* This is where the **tails** come into play.

In this workshop, we will use simple Psi4 calculations and data visualization to interactively explore several questions related to the short- and long-range behavior of stuff:

0. How to interpret pictures of orbitals?

1. Slater versus Gaussian basis functions: why do we prefer the latter?

2. Short-range correlation cusp and the explicitly correlated F12 theory:
   - Does commonly used F12 (Slater correlation factor expanded in Gaussians) reproduce the cusp?
   - Does F12 improve basis set convergence?

3. Long-range interaction energy on the example of two rare gas atoms:
   - How do long-range results depend on the cardinality of the basis set?
   - Does F12 (a method designed to fix the short range) improve the long range?
   - Does the addition of midbond functions improve the large-R results?
   - Does the addition of midbond functions improve the $C_6$ constant?
   - Basis set superposition error, counterpoise correction, and midbonds: how to do the best job and how to just avoid a disaster?

4. Density tails and their role in intermolecular interactions
   - How do electron densities and their overlap decay at large R? How does the basis set influence this decay?
   - How does density overlap relate to first-order exchange energy?
   - What is charge penetration and why should we care?

5. If time allows, some bonus topics related to regularization aka range separation of the potential.

Please be mindful that this is an alpha version of the workshop - constructive feedback will be highly appreciated. First and foremost, have fun! 

Konrad Patkowski, Auburn University (patkowsk@auburn.edu; ORCID: 0000-0002-4468-207X)

In [ ]:
# Let's start by importing Psi4, NumPy, SciPy, and Matplotlib

%matplotlib notebook
import time
import numpy as np
import scipy
from scipy.optimize import *
np.set_printoptions(precision=5, linewidth=200, threshold=2000, suppress=True)
import psi4
import matplotlib.pyplot as plt

# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
psi4.core.set_output_file('output.dat', False)

numpy_memory = 2

psi4.set_options({'basis': 'aug-cc-pvtz',
              'df_basis_scf': 'aug-cc-pvtz-ri',
              'df_basis_mp2': 'aug-cc-pvtz-ri',
              'e_convergence': 1e-10,
              'd_convergence': 1e-10,
              'ints_tolerance': 1e-18})

# 0. Visualizing p orbitals

In [ ]:
def r(x,y,z):
    return np.sqrt(x*x+y*y+z*z)

def orb2pz(x,y,z):
    return z*np.exp(-r(x,y,z)/2)/np.sqrt(32*np.pi)

def orb3pz(x,y,z):
    return z*(6-r(x,y,z))*np.exp(-r(x,y,z)/3)*np.sqrt(2/np.pi)/81

The following visualization of hydrogen-atom p orbitals has been taken from LibreTexts Chemistry:

[https://chem.libretexts.org/Ancillary_Materials/Interactive_Applications/Jupyter_Notebooks/Hydrogen_Orbitals_(Python_Notebook)](https://chem.libretexts.org/Ancillary_Materials/Interactive_Applications/Jupyter_Notebooks/Hydrogen_Orbitals_(Python_Notebook))

In [ ]:
from matplotlib.widgets import Slider, Button, RadioButtons
import scipy.special
from scipy.special import sph_harm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.colors import ListedColormap
import skimage
from skimage import measure
  
dz=0.5
zmin=-10
zmax=10
x = np.arange(zmin,zmax,dz)
y = np.arange(zmin,zmax,dz)
z = np.arange(zmin,zmax,dz)
X,Y,Z = np.meshgrid(x,y,z) #X, Y, Z are 3d arrays that tell us the values of x, y, and z at every point in space

data = orb2pz(X,Y,Z)
data = abs(data)**2
R = np.sqrt(X**2+Y**2+Z**2)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim([0,len(x)])
ax.set_ylim([0,len(y)])
ax.set_zlim([0,len(z)])
max_val = np.max(data)

verts, faces, _, _ = measure.marching_cubes(data, max_val/2, spacing = (1,1,1))
result=ax.plot_trisurf(verts[:,0], verts[:,1], faces, verts[:,2], cmap ='Spectral', lw=0)

sli = Slider(plt.axes([0.25, 0.025, 0.65, 0.03]), "iso", 0, max_val, valinit=max_val/2)
ax.set_title("Hydrogen Orbital Isosurface (2p$_z$)")

def update(val):
    ax.clear()
    verts, faces, _, _ = measure.marching_cubes(data, sli.val, spacing = (1,1,1))
    result = ax.plot_trisurf(verts[:,0], verts[:,1], faces, verts[:,2], cmap ='Spectral', lw=0)
    ax.set_xlim([0,len(x)])
    ax.set_ylim([0,len(y)])
    ax.set_zlim([0,len(z)])
    ax.set_title("Hydrogen Orbital Isosurface (2p$_z$)")
    plt.show()
           
sli.on_changed(update)
plt.show()

In [ ]:
zmin=-15
zmax=15
x = np.arange(zmin,zmax,dz)
y = np.arange(zmin,zmax,dz)
z = np.arange(zmin,zmax,dz)
X,Y,Z = np.meshgrid(x,y,z) #X, Y, Z are 3d arrays that tell us the values of x, y, and z at every point in space

R = np.sqrt(X**2+Y**2+Z**2)
data = orb3pz(X,Y,Z)
data = abs(data)**2

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim([0,len(x)])
ax.set_ylim([0,len(y)])
ax.set_zlim([0,len(z)])
max_val = np.max(data)

verts, faces, _, _ = measure.marching_cubes(data, max_val/2, spacing = (1,1,1))
result=ax.plot_trisurf(verts[:,0], verts[:,1], faces, verts[:,2], cmap ='Spectral', lw=0)

sli = Slider(plt.axes([0.25, 0.025, 0.65, 0.03]), "iso", 0, max_val, valinit=max_val/2)
ax.set_title("Hydrogen Orbital Isosurface (3p$_z$)")

def update(val):
    ax.clear()
    verts, faces, _, _ = measure.marching_cubes(data, sli.val, spacing = (1,1,1))
    result = ax.plot_trisurf(verts[:,0], verts[:,1], faces, verts[:,2], cmap ='Spectral', lw=0)
    ax.set_xlim([0,len(x)])
    ax.set_ylim([0,len(y)])
    ax.set_zlim([0,len(z)])
    ax.set_title("Hydrogen Orbital Isosurface (3p$_z$)")
    plt.show()
           
sli.on_changed(update)
plt.show()

# 1. Slater versus Gaussian basis functions: why do we prefer the latter?

The s-type Slater function centered at point $\mathbf{A}$ with exponent $\alpha$ is defined as
$$ \psi_{Slater}(\mathbf{r}) = N e^{-\alpha|\mathbf{r}-\mathbf{A}|} $$

while an s-type Gaussian function at the same center and exponent looks like this:
$$ \psi_{Gaussian}(\mathbf{r}) = N e^{-\alpha|\mathbf{r}-\mathbf{A}|^2} $$

We will look at the contour plots of those functions in the $xy$ plane.

In [ ]:
def slater_s_function(x,y,A,alpha):
# unnormalized
    distance = np.sqrt((x-A[0])*(x-A[0])+(y-A[1])*(y-A[1]))
    return 10.0*np.exp(-alpha*distance)

def gaussian_s_function(x,y,A,alpha):
# unnormalized
    distance = np.sqrt((x-A[0])*(x-A[0])+(y-A[1])*(y-A[1]))
    return 5.0*np.exp(-alpha*distance*distance)

A = np.array([0.0,0.0,0.0])
B = np.array([3.0,0.0,0.0])
alpha1 = 0.2
alpha2 = 0.4
x = np.linspace(-5.0, 8.0, 131) # 131 is the total number of points generated
y = np.linspace(-5.0, 5.0, 101)
Xgrid,Ygrid = np.meshgrid(x,y)

slater1 = slater_s_function(Xgrid,Ygrid,A,alpha1)
slater2 = slater_s_function(Xgrid,Ygrid,B,alpha2)
gaussian1 = gaussian_s_function(Xgrid,Ygrid,A,alpha1)
gaussian2 = gaussian_s_function(Xgrid,Ygrid,B,alpha2)
print(slater1.shape)
print(Xgrid.shape)
print(Ygrid.shape)

We defined Slater and Gaussian basis functions centered at two different point in space. What does their product look like (and why do we care)?

In [ ]:
plt.close()
fig = plt.figure(figsize=(8,12))

ax = fig.add_subplot(3, 2, 1, projection="3d")
ax.plot_surface(Xgrid, Ygrid, slater1, cmap="summer")

ax = fig.add_subplot(3, 2, 2)
ax.contourf(x, y, slater1, cmap="summer")

ax = fig.add_subplot(3, 2, 3, projection="3d")
ax.plot_surface(Xgrid, Ygrid, slater2, cmap="summer")

ax = fig.add_subplot(3, 2, 4)
ax.contourf(x, y, slater2, cmap="summer")

ax = fig.add_subplot(3, 2, 5, projection="3d")
ax.plot_surface(Xgrid, Ygrid, slater1*slater2, cmap="summer")

ax = fig.add_subplot(3, 2, 6)
ax.contourf(x, y, slater1*slater2, cmap="summer")

In [ ]:
plt.close()
fig = plt.figure(figsize=(8,12))

ax = fig.add_subplot(3, 2, 1, projection="3d")
ax.plot_surface(Xgrid, Ygrid, gaussian1, cmap="summer")

ax = fig.add_subplot(3, 2, 2)
ax.contourf(x, y, gaussian1, cmap="summer")

ax = fig.add_subplot(3, 2, 3, projection="3d")
ax.plot_surface(Xgrid, Ygrid, gaussian2, cmap="summer")

ax = fig.add_subplot(3, 2, 4)
ax.contourf(x, y, gaussian2, cmap="summer")

ax = fig.add_subplot(3, 2, 5, projection="3d")
ax.plot_surface(Xgrid, Ygrid, gaussian1*gaussian2, cmap="summer")

ax = fig.add_subplot(3, 2, 6)
ax.contourf(x, y, gaussian1*gaussian2, cmap="summer")

# 2. Slater and Gaussian correlation factors in the F12 theory

Correlated wavefunction calculations (MP2, CCSD(T), SAPT2+(3), ...) using standard bases of Gaussian one-electron functions converge slowly to the complete basis set (CBS) limit because they cannot properly describe the *correlation cusp* (a derivative discontinuity when two electrons coalesce, $r_{ij}=|\mathbf{r}_i-\mathbf{r}_j|\to 0$). The CBS convergence can be greatly sped up by the explicitly correlated F12 approach where an explicit dependence on the interelectronic distances $r_{ij}$ is thrown into the wavefunction.

A super-accurate and super-expensive explicitly correlated theory would explicitly (variationally) optimize the form of the pair function dependence on the interelectronic distance $r_{12}$. However, the philosophy of the F12 theory is quite different: we need a method that is nearly as efficient as its non-F12 variant, and *we can make drastic approximations because any dependence on $r_{12}$ helps*. Practice has shown that a Slater correlation factor
$$ F_{12}(\mathbf{r}_{12}) = \frac{1}{\beta} e^{-\beta r_{12}} $$

is a practical and well performing choice, but the new integrals are quite cumbersome. 

How can we simplify the integrals? Just like before - Gaussian functions to the rescue! Let's approximate the Slater correlation factor with a linear combination of Gaussians:

$$ F_{12}(\mathbf{r}_{12}) \approx \sum_{i=1}^n c_i e^{-\alpha_i r_{12}^2} $$

Let's take a look at this approximation.

In [ ]:
def slater_corr_factor(r12):
    return np.exp(-r12)

def gauss6_fit_slater(r12):
# exponents and coefficients from Erica Mitchell's pull request in Psi4 (also ORCA and MPQC)
    coeffs = np.array([0.31442480597241274, 0.30369575353387201, 0.16806968430232927, 
                       0.098115812152857612, 0.060246640234342785, 0.037263541968504843])
    exps = np.array([0.22085085450735284, 1.0040191632019282, 3.6212173098378728, 
                     12.162483236221904, 45.855332448029337, 254.23460688554644])
    sum = 0.0
    for i in range(6):
        sum += coeffs[i] * np.exp(-exps[i] * r12 * r12)
    return sum

r12s = np.linspace(0.0,5.0,1001)
slaters = slater_corr_factor(r12s)
gauss6s = gauss6_fit_slater(r12s)

plt.close()
plt.plot(r12s,slaters,'g',linestyle='-',label='Slater corr. factor')
plt.plot(r12s,gauss6s,'b',linestyle='-',label='6-Gaussian fit')
plt.legend(loc='upper right')
plt.xlim((0.0,5.0))
plt.ylim((0.0,1.1))
plt.show()

# 3. Basis sets and midbonds

We will compute the MP2 interaction energies for He$\cdots$He and Ne$\cdots$Ne in different basis sets. The MP2 interaction energy is defined in the standard *supermolecular* way:
$$ E_{\rm int}^{\rm MP2} = E_{\rm AB}^{\rm MP2} - E_{\rm A}^{\rm MP2} - E_{\rm B}^{\rm MP2} $$

The only degree of freedom is the interatomic distance $R$, so we will obtain potential energy curves as functions of $R$.

Some choices that we need to make:
1. What basis set to use?
2. Do we compute conventional MP2 or explicitly correlated MP2-F12 (not yet available in the official Psi4 version)?
3. In the calculations of $E_{\rm A}^{\rm MP2}$ and $E_{\rm B}^{\rm MP2}$, do we use the full basis set of the complex *(counterpoise corrected calculation)* or only the basis set of this fragment *(counterpoise uncorrected calculation)*?
4. Do we add some extra basis functions halfway between the interacting atoms? If yes, then what functions (the simple choice we will make is pretend there's a "ghost" hydrogen atom out there, and use hydrogen basis functions)?

Let's explore some of these choices.

In [ ]:
distances = [4.0,4.5,5.0,5.3,5.6,6.0,6.5,7.0,8.0,9.0,10.0,12.0,14.0,16.0,18.0,20.0]

In [ ]:
def run_curve_rg2_cp(atom,distances,midbond=False):
    ehf = np.zeros((len(distances),3))
    emp2 = np.zeros((len(distances),3))
    for i in range(len(distances)):
        if (midbond):
            dimer = psi4.geometry(atom+""" 0.0 0.0 0.0
            """+atom+""" 0.0 0.0 """+str(distances[i])+"""
            Gh(H) 0.0 0.0 """+str(0.5*distances[i])+"""
            units bohr
            symmetry c1
            """)
        else:
            dimer = psi4.geometry(atom+""" 0.0 0.0 0.0
            """+atom+""" 0.0 0.0 """+str(distances[i])+"""
            units bohr
            symmetry c1
            """)

        psi4.energy('mp2')   #HF will be calculated along the way
        ehf[i,0] = psi4.variable('HF TOTAL ENERGY')
        emp2[i,0] = psi4.variable('MP2 TOTAL ENERGY')
        psi4.core.clean()

        if (midbond):
            monomerA = psi4.geometry(atom+""" 0.0 0.0 0.0
            Gh("""+atom+""") 0.0 0.0 """+str(distances[i])+"""
            Gh(H) 0.0 0.0 """+str(0.5*distances[i])+"""
            units bohr
            symmetry c1
            """)
        else:
            monomerA = psi4.geometry(atom+""" 0.0 0.0 0.0
            Gh("""+atom+""") 0.0 0.0 """+str(distances[i])+"""
            units bohr
            symmetry c1
            """)

        psi4.energy('mp2')   #HF will be calculated along the way
        ehf[i,1] = psi4.variable('HF TOTAL ENERGY')
        emp2[i,1] = psi4.variable('MP2 TOTAL ENERGY')
        psi4.core.clean()

        if (midbond):
            monomerB = psi4.geometry("Gh("+atom+""") 0.0 0.0 0.0
            """+atom+""" 0.0 0.0 """+str(distances[i])+"""
            Gh(H) 0.0 0.0 """+str(0.5*distances[i])+"""
            units bohr
            symmetry c1
            """)
        else:
            monomerB = psi4.geometry("Gh("+atom+""") 0.0 0.0 0.0
            """+atom+""" 0.0 0.0 """+str(distances[i])+"""
            units bohr
            symmetry c1
            """)

        psi4.energy('mp2')   #HF will be calculated along the way
        ehf[i,2] = psi4.variable('HF TOTAL ENERGY')
        emp2[i,2] = psi4.variable('MP2 TOTAL ENERGY')
        psi4.core.clean()
    return(ehf[:],emp2[:])

In [ ]:
def run_curve_rg2_noncp(atom,distances,midbond=False):
    ehf = np.zeros((len(distances),3))
    emp2 = np.zeros((len(distances),3))
    for i in range(len(distances)):
        if (midbond):
            dimer = psi4.geometry(atom+""" 0.0 0.0 0.0
            """+atom+""" 0.0 0.0 """+str(distances[i])+"""
            Gh(H) 0.0 0.0 """+str(0.5*distances[i])+"""
            units bohr
            symmetry c1
            """)
        else:
            dimer = psi4.geometry(atom+""" 0.0 0.0 0.0
            """+atom+""" 0.0 0.0 """+str(distances[i])+"""
            units bohr
            symmetry c1
            """)

        psi4.energy('mp2')   #HF will be calculated along the way
        ehf[i,0] = psi4.variable('HF TOTAL ENERGY')
        emp2[i,0] = psi4.variable('MP2 TOTAL ENERGY')
        psi4.core.clean()

        monomerA = psi4.geometry(atom+""" 0.0 0.0 0.0
        units bohr
        symmetry c1
        """)

        psi4.energy('mp2')   #HF will be calculated along the way
        ehf[i,1] = psi4.variable('HF TOTAL ENERGY')
        emp2[i,1] = psi4.variable('MP2 TOTAL ENERGY')
        psi4.core.clean()

        monomerB = psi4.geometry(atom+""" 0.0 0.0 """+str(distances[i])+"""
        units bohr
        symmetry c1
        """)

        psi4.energy('mp2')   #HF will be calculated along the way
        ehf[i,2] = psi4.variable('HF TOTAL ENERGY')
        emp2[i,2] = psi4.variable('MP2 TOTAL ENERGY')
        psi4.core.clean()
    return(ehf[:],emp2[:])

In [ ]:
psi4.set_options({'basis': 'aug-cc-pvdz',
              'df_basis_scf': 'aug-cc-pvtz-ri',
              'df_basis_mp2': 'aug-cc-pvtz-ri'})
(hf_adz_cp,mp2_adz_cp) = run_curve_rg2_cp("He",distances)
(hf_adz_noncp,mp2_adz_noncp) = run_curve_rg2_noncp("He",distances)
(Mhf_adz_cp,Mmp2_adz_cp) = run_curve_rg2_cp("He",distances,midbond=True)
(Mhf_adz_noncp,Mmp2_adz_noncp) = run_curve_rg2_noncp("He",distances,midbond=True)
psi4.set_options({'basis': 'aug-cc-pvtz',
              'df_basis_scf': 'aug-cc-pvtz-ri',
              'df_basis_mp2': 'aug-cc-pvtz-ri'})
(hf_atz_cp,mp2_atz_cp) = run_curve_rg2_cp("He",distances)
(hf_atz_noncp,mp2_atz_noncp) = run_curve_rg2_noncp("He",distances)
(Mhf_atz_cp,Mmp2_atz_cp) = run_curve_rg2_cp("He",distances,midbond=True)
(Mhf_atz_noncp,Mmp2_atz_noncp) = run_curve_rg2_noncp("He",distances,midbond=True)
psi4.set_options({'basis': 'aug-cc-pvqz',
              'df_basis_scf': 'aug-cc-pvqz-ri',
              'df_basis_mp2': 'aug-cc-pvqz-ri'})
(hf_aqz_cp,mp2_aqz_cp) = run_curve_rg2_cp("He",distances)
(hf_aqz_noncp,mp2_aqz_noncp) = run_curve_rg2_noncp("He",distances)
(Mhf_aqz_cp,Mmp2_aqz_cp) = run_curve_rg2_cp("He",distances,midbond=True)
(Mhf_aqz_noncp,Mmp2_aqz_noncp) = run_curve_rg2_noncp("He",distances,midbond=True)

In [ ]:
eintmp2_adz_cp = (mp2_adz_cp[:,0] - mp2_adz_cp[:,1] - mp2_adz_cp[:,2]) * 627.509
eintmp2_adz_noncp = (mp2_adz_noncp[:,0] - mp2_adz_noncp[:,1] - mp2_adz_noncp[:,2]) * 627.509
eintmp2_atz_cp = (mp2_atz_cp[:,0] - mp2_atz_cp[:,1] - mp2_atz_cp[:,2]) * 627.509
eintmp2_atz_noncp = (mp2_atz_noncp[:,0] - mp2_atz_noncp[:,1] - mp2_atz_noncp[:,2]) * 627.509
eintmp2_aqz_cp = (mp2_aqz_cp[:,0] - mp2_aqz_cp[:,1] - mp2_aqz_cp[:,2]) * 627.509
eintmp2_aqz_noncp = (mp2_aqz_noncp[:,0] - mp2_aqz_noncp[:,1] - mp2_aqz_noncp[:,2]) * 627.509
Meintmp2_adz_cp = (Mmp2_adz_cp[:,0] - Mmp2_adz_cp[:,1] - Mmp2_adz_cp[:,2]) * 627.509
Meintmp2_adz_noncp = (Mmp2_adz_noncp[:,0] - Mmp2_adz_noncp[:,1] - Mmp2_adz_noncp[:,2]) * 627.509
Meintmp2_atz_cp = (Mmp2_atz_cp[:,0] - Mmp2_atz_cp[:,1] - Mmp2_atz_cp[:,2]) * 627.509
Meintmp2_atz_noncp = (Mmp2_atz_noncp[:,0] - Mmp2_atz_noncp[:,1] - Mmp2_atz_noncp[:,2]) * 627.509
Meintmp2_aqz_cp = (Mmp2_aqz_cp[:,0] - Mmp2_aqz_cp[:,1] - Mmp2_aqz_cp[:,2]) * 627.509
Meintmp2_aqz_noncp = (Mmp2_aqz_noncp[:,0] - Mmp2_aqz_noncp[:,1] - Mmp2_aqz_noncp[:,2]) * 627.509

plt.close()
plt.plot(distances,eintmp2_adz_cp,'b+',linestyle='-',label='He-He MP2/aDZ CP')
plt.plot(distances,eintmp2_adz_noncp,'bo',linestyle='--',label='He-He MP2/aDZ nonCP')
plt.plot(distances,eintmp2_atz_cp,'r+',linestyle='-',label='He-He MP2/aTZ CP')
plt.plot(distances,eintmp2_atz_noncp,'ro',linestyle='--',label='He-He MP2/aTZ nonCP')
plt.plot(distances,eintmp2_aqz_cp,'g+',linestyle='-',label='He-He MP2/aQZ CP')
plt.plot(distances,eintmp2_aqz_noncp,'go',linestyle='--',label='He-He MP2/aQZ nonCP')
plt.plot(distances,Meintmp2_adz_cp,'k+',linestyle='-',label='He-He MP2/aDZ+(aDZ) CP')
plt.plot(distances,Meintmp2_adz_noncp,'ko',linestyle='--',label='He-He MP2/aDZ+(aDZ) nonCP')
plt.plot(distances,Meintmp2_atz_cp,'c+',linestyle='-',label='He-He MP2/aTZ+(aTZ) CP')
plt.plot(distances,Meintmp2_atz_noncp,'co',linestyle='--',label='He-He MP2/aTZ+(aTZ) nonCP')
plt.plot(distances,Meintmp2_aqz_cp,'m+',linestyle='-',label='He-He MP2/aQZ+(aQZ) CP')
plt.plot(distances,Meintmp2_aqz_noncp,'mo',linestyle='--',label='He-He MP2/aQZ+(aQZ) nonCP')
plt.legend(loc='upper right')
plt.ylim((-0.025,0.025))
plt.show()


We will now move on to the neon dimer and compute the same set of MP2 potential energy curves. When we ran this notebook at the workshop, we ran into some issues at long range which were tracked down to the errors of the density-fitting approximation in DF-MP2. Therefore, we will first run DF-MP2, but then also conventional MP2 (requested by the options `psi4.set_options({'scf_type': 'pk', 'mp2_type': 'conv'})`). The latter might take an hour or so (the density fitting approximation really speeds things up!) but we will have a complete picture.

In [ ]:
psi4.set_options({'basis': 'aug-cc-pvdz',
              'df_basis_scf': 'aug-cc-pvtz-ri',
              'df_basis_mp2': 'aug-cc-pvtz-ri'})
(hf_adz_cp,mp2_adz_cp) = run_curve_rg2_cp("Ne",distances)
(hf_adz_noncp,mp2_adz_noncp) = run_curve_rg2_noncp("Ne",distances)
(Mhf_adz_cp,Mmp2_adz_cp) = run_curve_rg2_cp("Ne",distances,midbond=True)
(Mhf_adz_noncp,Mmp2_adz_noncp) = run_curve_rg2_noncp("Ne",distances,midbond=True)
psi4.set_options({'basis': 'aug-cc-pvtz',
              'df_basis_scf': 'aug-cc-pvtz-ri',
              'df_basis_mp2': 'aug-cc-pvtz-ri'})
(hf_atz_cp,mp2_atz_cp) = run_curve_rg2_cp("Ne",distances)
(hf_atz_noncp,mp2_atz_noncp) = run_curve_rg2_noncp("Ne",distances)
(Mhf_atz_cp,Mmp2_atz_cp) = run_curve_rg2_cp("Ne",distances,midbond=True)
(Mhf_atz_noncp,Mmp2_atz_noncp) = run_curve_rg2_noncp("Ne",distances,midbond=True)
psi4.set_options({'basis': 'aug-cc-pvqz',
              'df_basis_scf': 'aug-cc-pvqz-ri',
              'df_basis_mp2': 'aug-cc-pvqz-ri'})
(hf_aqz_cp,mp2_aqz_cp) = run_curve_rg2_cp("Ne",distances)
(hf_aqz_noncp,mp2_aqz_noncp) = run_curve_rg2_noncp("Ne",distances)
(Mhf_aqz_cp,Mmp2_aqz_cp) = run_curve_rg2_cp("Ne",distances,midbond=True)
(Mhf_aqz_noncp,Mmp2_aqz_noncp) = run_curve_rg2_noncp("Ne",distances,midbond=True)

In [ ]:
psi4.set_options({'scf_type': 'pk',
                  'mp2_type': 'conv',
                  'basis': 'aug-cc-pvdz',
              'df_basis_scf': 'aug-cc-pvtz-ri',
              'df_basis_mp2': 'aug-cc-pvtz-ri'})
(chf_adz_cp,cmp2_adz_cp) = run_curve_rg2_cp("Ne",distances)
(chf_adz_noncp,cmp2_adz_noncp) = run_curve_rg2_noncp("Ne",distances)
(cMhf_adz_cp,cMmp2_adz_cp) = run_curve_rg2_cp("Ne",distances,midbond=True)
(cMhf_adz_noncp,cMmp2_adz_noncp) = run_curve_rg2_noncp("Ne",distances,midbond=True)
psi4.set_options({'basis': 'aug-cc-pvtz',
              'df_basis_scf': 'aug-cc-pvtz-ri',
              'df_basis_mp2': 'aug-cc-pvtz-ri'})
(chf_atz_cp,cmp2_atz_cp) = run_curve_rg2_cp("Ne",distances)
(chf_atz_noncp,cmp2_atz_noncp) = run_curve_rg2_noncp("Ne",distances)
(cMhf_atz_cp,cMmp2_atz_cp) = run_curve_rg2_cp("Ne",distances,midbond=True)
(cMhf_atz_noncp,cMmp2_atz_noncp) = run_curve_rg2_noncp("Ne",distances,midbond=True)
psi4.set_options({'basis': 'aug-cc-pvqz',
              'df_basis_scf': 'aug-cc-pvqz-ri',
              'df_basis_mp2': 'aug-cc-pvqz-ri'})
(chf_aqz_cp,cmp2_aqz_cp) = run_curve_rg2_cp("Ne",distances)
(chf_aqz_noncp,cmp2_aqz_noncp) = run_curve_rg2_noncp("Ne",distances)
(cMhf_aqz_cp,cMmp2_aqz_cp) = run_curve_rg2_cp("Ne",distances,midbond=True)
(cMhf_aqz_noncp,cMmp2_aqz_noncp) = run_curve_rg2_noncp("Ne",distances,midbond=True)
psi4.set_options({'scf_type': 'df',
                  'mp2_type': 'df'})

ceintmp2_adz_cp = (cmp2_adz_cp[:,0] - cmp2_adz_cp[:,1] - cmp2_adz_cp[:,2]) * 627.509
ceintmp2_adz_noncp = (cmp2_adz_noncp[:,0] - cmp2_adz_noncp[:,1] - cmp2_adz_noncp[:,2]) * 627.509
ceintmp2_atz_cp = (cmp2_atz_cp[:,0] - cmp2_atz_cp[:,1] - cmp2_atz_cp[:,2]) * 627.509
ceintmp2_atz_noncp = (cmp2_atz_noncp[:,0] - cmp2_atz_noncp[:,1] - cmp2_atz_noncp[:,2]) * 627.509
ceintmp2_aqz_cp = (cmp2_aqz_cp[:,0] - cmp2_aqz_cp[:,1] - cmp2_aqz_cp[:,2]) * 627.509
ceintmp2_aqz_noncp = (cmp2_aqz_noncp[:,0] - cmp2_aqz_noncp[:,1] - cmp2_aqz_noncp[:,2]) * 627.509
cMeintmp2_adz_cp = (cMmp2_adz_cp[:,0] - cMmp2_adz_cp[:,1] - cMmp2_adz_cp[:,2]) * 627.509
cMeintmp2_adz_noncp = (cMmp2_adz_noncp[:,0] - cMmp2_adz_noncp[:,1] - cMmp2_adz_noncp[:,2]) * 627.509
cMeintmp2_atz_cp = (cMmp2_atz_cp[:,0] - cMmp2_atz_cp[:,1] - cMmp2_atz_cp[:,2]) * 627.509
cMeintmp2_atz_noncp = (cMmp2_atz_noncp[:,0] - cMmp2_atz_noncp[:,1] - cMmp2_atz_noncp[:,2]) * 627.509
cMeintmp2_aqz_cp = (cMmp2_aqz_cp[:,0] - cMmp2_aqz_cp[:,1] - cMmp2_aqz_cp[:,2]) * 627.509
cMeintmp2_aqz_noncp = (cMmp2_aqz_noncp[:,0] - cMmp2_aqz_noncp[:,1] - cMmp2_aqz_noncp[:,2]) * 627.509

In [ ]:
eintmp2_adz_cp = (mp2_adz_cp[:,0] - mp2_adz_cp[:,1] - mp2_adz_cp[:,2]) * 627.509
eintmp2_adz_noncp = (mp2_adz_noncp[:,0] - mp2_adz_noncp[:,1] - mp2_adz_noncp[:,2]) * 627.509
eintmp2_atz_cp = (mp2_atz_cp[:,0] - mp2_atz_cp[:,1] - mp2_atz_cp[:,2]) * 627.509
eintmp2_atz_noncp = (mp2_atz_noncp[:,0] - mp2_atz_noncp[:,1] - mp2_atz_noncp[:,2]) * 627.509
eintmp2_aqz_cp = (mp2_aqz_cp[:,0] - mp2_aqz_cp[:,1] - mp2_aqz_cp[:,2]) * 627.509
eintmp2_aqz_noncp = (mp2_aqz_noncp[:,0] - mp2_aqz_noncp[:,1] - mp2_aqz_noncp[:,2]) * 627.509
Meintmp2_adz_cp = (Mmp2_adz_cp[:,0] - Mmp2_adz_cp[:,1] - Mmp2_adz_cp[:,2]) * 627.509
Meintmp2_adz_noncp = (Mmp2_adz_noncp[:,0] - Mmp2_adz_noncp[:,1] - Mmp2_adz_noncp[:,2]) * 627.509
Meintmp2_atz_cp = (Mmp2_atz_cp[:,0] - Mmp2_atz_cp[:,1] - Mmp2_atz_cp[:,2]) * 627.509
Meintmp2_atz_noncp = (Mmp2_atz_noncp[:,0] - Mmp2_atz_noncp[:,1] - Mmp2_atz_noncp[:,2]) * 627.509
Meintmp2_aqz_cp = (Mmp2_aqz_cp[:,0] - Mmp2_aqz_cp[:,1] - Mmp2_aqz_cp[:,2]) * 627.509
Meintmp2_aqz_noncp = (Mmp2_aqz_noncp[:,0] - Mmp2_aqz_noncp[:,1] - Mmp2_aqz_noncp[:,2]) * 627.509

plt.close()
plt.plot(distances,eintmp2_adz_cp,'b+',linestyle='-',label='Ne-Ne DF-MP2/aDZ CP')
plt.plot(distances,eintmp2_adz_noncp,'bo',linestyle='--',label='Ne-Ne DF-MP2/aDZ nonCP')
plt.plot(distances,eintmp2_atz_cp,'r+',linestyle='-',label='Ne-Ne DF-MP2/aTZ CP')
plt.plot(distances,eintmp2_atz_noncp,'ro',linestyle='--',label='Ne-Ne DF-MP2/aTZ nonCP')
plt.plot(distances,eintmp2_aqz_cp,'g+',linestyle='-',label='Ne-Ne DF-MP2/aQZ CP')
plt.plot(distances,eintmp2_aqz_noncp,'go',linestyle='--',label='Ne-Ne DF-MP2/aQZ nonCP')
plt.plot(distances,Meintmp2_adz_cp,'k+',linestyle='-',label='Ne-Ne DF-MP2/aDZ+(aDZ) CP')
plt.plot(distances,Meintmp2_adz_noncp,'ko',linestyle='--',label='Ne-Ne DF-MP2/aDZ+(aDZ) nonCP')
plt.plot(distances,Meintmp2_atz_cp,'c+',linestyle='-',label='Ne-Ne DF-MP2/aTZ+(aTZ) CP')
plt.plot(distances,Meintmp2_atz_noncp,'co',linestyle='--',label='Ne-Ne DF-MP2/aTZ+(aTZ) nonCP')
plt.plot(distances,Meintmp2_aqz_cp,'m+',linestyle='-',label='Ne-Ne DF-MP2/aQZ+(aQZ) CP')
plt.plot(distances,Meintmp2_aqz_noncp,'mo',linestyle='--',label='Ne-Ne DF-MP2/aQZ+(aQZ) nonCP')
plt.legend(loc='upper right')
plt.xlim((4.0,14.0))
plt.ylim((-0.15,0.03))
plt.show()

In [ ]:
plt.close()
plt.plot(distances,ceintmp2_adz_cp,'b+',linestyle='-',label='Ne-Ne MP2/aDZ CP')
plt.plot(distances,ceintmp2_adz_noncp,'bo',linestyle='--',label='Ne-Ne MP2/aDZ nonCP')
plt.plot(distances,ceintmp2_atz_cp,'r+',linestyle='-',label='Ne-Ne MP2/aTZ CP')
plt.plot(distances,ceintmp2_atz_noncp,'ro',linestyle='--',label='Ne-Ne MP2/aTZ nonCP')
plt.plot(distances,ceintmp2_aqz_cp,'g+',linestyle='-',label='Ne-Ne MP2/aQZ CP')
plt.plot(distances,ceintmp2_aqz_noncp,'go',linestyle='--',label='Ne-Ne MP2/aQZ nonCP')
plt.plot(distances,cMeintmp2_adz_cp,'k+',linestyle='-',label='Ne-Ne MP2/aDZ+(aDZ) CP')
plt.plot(distances,cMeintmp2_adz_noncp,'ko',linestyle='--',label='Ne-Ne MP2/aDZ+(aDZ) nonCP')
plt.plot(distances,cMeintmp2_atz_cp,'c+',linestyle='-',label='Ne-Ne MP2/aTZ+(aTZ) CP')
plt.plot(distances,cMeintmp2_atz_noncp,'co',linestyle='--',label='Ne-Ne MP2/aTZ+(aTZ) nonCP')
plt.plot(distances,cMeintmp2_aqz_cp,'m+',linestyle='-',label='Ne-Ne MP2/aQZ+(aQZ) CP')
plt.plot(distances,cMeintmp2_aqz_noncp,'mo',linestyle='--',label='Ne-Ne MP2/aQZ+(aQZ) nonCP')
plt.legend(loc='upper right')
plt.xlim((4.0,14.0))
plt.ylim((-0.15,0.03))
plt.show()

In [ ]:
plt.close()
plt.plot(distances,eintmp2_adz_cp,'b+',linestyle='-',label='Ne-Ne DF-MP2/aDZ CP')
plt.plot(distances,eintmp2_adz_noncp,'bo',linestyle='--',label='Ne-Ne DF-MP2/aDZ nonCP')
plt.plot(distances,eintmp2_atz_cp,'r+',linestyle='-',label='Ne-Ne DF-MP2/aTZ CP')
plt.plot(distances,eintmp2_atz_noncp,'ro',linestyle='--',label='Ne-Ne DF-MP2/aTZ nonCP')
plt.plot(distances,eintmp2_aqz_cp,'g+',linestyle='-',label='Ne-Ne DF-MP2/aQZ CP')
plt.plot(distances,eintmp2_aqz_noncp,'go',linestyle='--',label='Ne-Ne DF-MP2/aQZ nonCP')
plt.plot(distances,Meintmp2_adz_cp,'k+',linestyle='-',label='Ne-Ne DF-MP2/aDZ+(aDZ) CP')
plt.plot(distances,Meintmp2_atz_cp,'c+',linestyle='-',label='Ne-Ne DF-MP2/aTZ+(aTZ) CP')
plt.plot(distances,Meintmp2_aqz_cp,'m+',linestyle='-',label='Ne-Ne DF-MP2/aQZ+(aQZ) CP')
plt.legend(loc='lower right')
plt.xlim((4.0,12.0))
plt.ylim((-0.10,0.0))
plt.show()

In [ ]:
plt.close()
plt.plot(distances,ceintmp2_adz_cp,'b+',linestyle='-',label='Ne-Ne MP2/aDZ CP')
plt.plot(distances,ceintmp2_adz_noncp,'bo',linestyle='--',label='Ne-Ne MP2/aDZ nonCP')
plt.plot(distances,ceintmp2_atz_cp,'r+',linestyle='-',label='Ne-Ne MP2/aTZ CP')
plt.plot(distances,ceintmp2_atz_noncp,'ro',linestyle='--',label='Ne-Ne MP2/aTZ nonCP')
plt.plot(distances,ceintmp2_aqz_cp,'g+',linestyle='-',label='Ne-Ne MP2/aQZ CP')
plt.plot(distances,ceintmp2_aqz_noncp,'go',linestyle='--',label='Ne-Ne MP2/aQZ nonCP')
plt.plot(distances,cMeintmp2_adz_cp,'k+',linestyle='-',label='Ne-Ne MP2/aDZ+(aDZ) CP')
plt.plot(distances,cMeintmp2_atz_cp,'c+',linestyle='-',label='Ne-Ne MP2/aTZ+(aTZ) CP')
plt.plot(distances,cMeintmp2_aqz_cp,'m+',linestyle='-',label='Ne-Ne MP2/aQZ+(aQZ) CP')
plt.legend(loc='lower right')
plt.xlim((4.0,12.0))
plt.ylim((-0.10,0.0))
plt.show()

We see that at distances around the van der Waals minimum the conventional MP2 and DF-MP2 interaction energies agree with each other very well, but what about larger distances?

In [ ]:
plt.close()
#plt.plot(distances,eintmp2_adz_cp,'b+',linestyle='-',label='Ne-Ne MP2/aDZ CP')
plt.plot(distances,eintmp2_atz_cp,'r+',linestyle='-',label='Ne-Ne DF-MP2/aTZ CP')
plt.plot(distances,eintmp2_aqz_cp,'g+',linestyle='-',label='Ne-Ne DF-MP2/aQZ CP')
#plt.plot(distances,Meintmp2_adz_cp,'ko',linestyle='--',label='Ne-Ne MP2/aDZ+(aDZ) CP')
plt.plot(distances,Meintmp2_atz_cp,'co',linestyle='--',label='Ne-Ne DF-MP2/aTZ+(aTZ) CP')
plt.plot(distances,Meintmp2_aqz_cp,'mo',linestyle='--',label='Ne-Ne DF-MP2/aQZ+(aQZ) CP')
plt.legend(loc='lower right')
plt.xlim((10.0,20.0))
plt.ylim((-0.005,0.0))
plt.show()

In [ ]:
plt.close()
#plt.plot(distances,eintmp2_adz_cp,'b+',linestyle='-',label='Ne-Ne MP2/aDZ CP')
plt.plot(distances,ceintmp2_atz_cp,'r+',linestyle='-',label='Ne-Ne Conv. MP2/aTZ CP')
plt.plot(distances,ceintmp2_aqz_cp,'g+',linestyle='-',label='Ne-Ne Conv. MP2/aQZ CP')
#plt.plot(distances,Meintmp2_adz_cp,'ko',linestyle='--',label='Ne-Ne MP2/aDZ+(aDZ) CP')
plt.plot(distances,cMeintmp2_atz_cp,'co',linestyle='--',label='Ne-Ne Conv. MP2/aTZ+(aTZ) CP')
plt.plot(distances,cMeintmp2_aqz_cp,'mo',linestyle='--',label='Ne-Ne Conv. MP2/aQZ+(aQZ) CP')
plt.legend(loc='lower right')
plt.xlim((10.0,20.0))
plt.ylim((-0.005,0.0))
plt.show()

Well, at these separations the DF approximation clearly ran into some issues! On the other hand, the conventional MP2 values make perfect sense: the neon-neon interaction energy at long range is nearly all dispersion, and MP2 dispersion is closely related to the leading order $E^{(20)}_{\rm disp}$ dispersion energy from SAPT. $E^{(20)}_{\rm disp}$ is a variational quantity, meaning that its value becomes better (more negative) as the basis set is increased. This is exactly what we observe.

It will be useful to compare the MP2 numbers with the SAPT0 dispersion explicitly, so we will run the SAPT0 curves.  Our SAPT0 calculation uses DF, which is still OK here because SAPT calculates the dispersion energy directly, without subtraction.

In [ ]:
def run_curve_rg2_sapt0(atom,distances,midbond=False):
    edisp_rg2 = np.zeros((len(distances)))
    esapt_rg2 = np.zeros((len(distances)))
    for i in range(len(distances)):
        if (midbond):
            dimer = psi4.geometry("""0 1
            """+atom+""" 0.0 0.0 0.0
            --
            0 1
            """+atom+""" 0.0 0.0 """+str(distances[i])+"""
            --
            Gh(H) 0.0 0.0 """+str(0.5*distances[i])+"""
            units bohr
            symmetry c1
            """)
        else:
            dimer = psi4.geometry("""0 1
            """+atom+""" 0.0 0.0 0.0
            --
            0 1
            """+atom+""" 0.0 0.0 """+str(distances[i])+"""
            units bohr
            symmetry c1
            """)

        psi4.energy('sapt0')   #HF will be calculated along the way
        edisp_rg2[i] = psi4.variable('SAPT DISP ENERGY') * 627.509
        esapt_rg2[i] = psi4.variable('SAPT TOTAL ENERGY') * 627.509
        psi4.core.clean()


    return(edisp_rg2[:],esapt_rg2[:])

In [ ]:
psi4.set_options({'basis': 'aug-cc-pvtz',
              'df_basis_scf': 'aug-cc-pvtz-ri',
              'df_basis_mp2': 'aug-cc-pvtz-ri',
              'df_basis_sapt': 'aug-cc-pvtz-ri',})
(disp_atz_cp,sapt0_atz_cp) = run_curve_rg2_sapt0("Ne",distances)
(Mdisp_atz_cp,Msapt0_atz_cp) = run_curve_rg2_sapt0("Ne",distances,midbond=True)
psi4.set_options({'basis': 'aug-cc-pvqz',
              'df_basis_scf': 'aug-cc-pvqz-ri',
              'df_basis_mp2': 'aug-cc-pvqz-ri',
              'df_basis_sapt': 'aug-cc-pvqz-ri',})
(disp_aqz_cp,sapt0_aqz_cp) = run_curve_rg2_sapt0("Ne",distances)
(Mdisp_aqz_cp,Msapt0_aqz_cp) = run_curve_rg2_sapt0("Ne",distances,midbond=True)

In [ ]:
plt.close()
plt.plot(distances,eintmp2_atz_cp,'r',linestyle='-',label='Ne-Ne DF-MP2/aTZ CP')
plt.plot(distances,eintmp2_aqz_cp,'g',linestyle='-',label='Ne-Ne DF-MP2/aQZ CP')
plt.plot(distances,Meintmp2_atz_cp,'c',linestyle='--',label='Ne-Ne DF-MP2/aTZ+(aTZ) CP')
plt.plot(distances,Meintmp2_aqz_cp,'m',linestyle='--',label='Ne-Ne DF-MP2/aQZ+(aQZ) CP')
plt.plot(distances,disp_atz_cp,'r+',linestyle=' ',label='Ne-Ne SAPT0 disp/aTZ CP')
plt.plot(distances,disp_aqz_cp,'go',linestyle=' ',label='Ne-Ne SAPT0 disp/aQZ CP')
plt.plot(distances,Mdisp_atz_cp,'c^',linestyle=' ',label='Ne-Ne SAPT0 disp/aTZ+(aTZ) CP')
plt.plot(distances,Mdisp_aqz_cp,'m*',linestyle=' ',label='Ne-Ne SAPT0 disp/aQZ+(aQZ) CP')
plt.legend(loc='lower right')
plt.xlim((11.0,20.0))
plt.ylim((-0.0013,0.0))
plt.show()

In [ ]:
plt.close()
plt.plot(distances,ceintmp2_atz_cp,'r',linestyle='-',label='Ne-Ne Conv. MP2/aTZ CP')
plt.plot(distances,ceintmp2_aqz_cp,'g',linestyle='-',label='Ne-Ne Conv. MP2/aQZ CP')
plt.plot(distances,cMeintmp2_atz_cp,'c',linestyle='--',label='Ne-Ne Conv. MP2/aTZ+(aTZ) CP')
plt.plot(distances,cMeintmp2_aqz_cp,'m',linestyle='--',label='Ne-Ne Conv. MP2/aQZ+(aQZ) CP')
plt.plot(distances,disp_atz_cp,'r+',linestyle=' ',label='Ne-Ne SAPT0 disp/aTZ CP')
plt.plot(distances,disp_aqz_cp,'go',linestyle=' ',label='Ne-Ne SAPT0 disp/aQZ CP')
plt.plot(distances,Mdisp_atz_cp,'c^',linestyle=' ',label='Ne-Ne SAPT0 disp/aTZ+(aTZ) CP')
plt.plot(distances,Mdisp_aqz_cp,'m*',linestyle=' ',label='Ne-Ne SAPT0 disp/aQZ+(aQZ) CP')
plt.legend(loc='lower right')
plt.xlim((11.0,20.0))
plt.ylim((-0.0013,0.0))
plt.show()

The agreement between conventional MP2 interaction energy and SAPT0 dispersion energy is perfect at those distances.

Let's go to even larger distances to see if midbond functions are still helping. Once again, conventional MP2 will take a while.

In [ ]:
distances_l = [20.0,22.5,25.0,27.5,30.0,32.5,35.0,37.5,40.0,42.5,45.0,47.5,50.0]

psi4.set_options({'basis': 'aug-cc-pvtz',
              'df_basis_scf': 'aug-cc-pvtz-ri',
              'df_basis_mp2': 'aug-cc-pvtz-ri'})
(hf_atz_cp_l,mp2_atz_cp_l) = run_curve_rg2_cp("Ne",distances_l)
(Mhf_atz_cp_l,Mmp2_atz_cp_l) = run_curve_rg2_cp("Ne",distances_l,midbond=True)
psi4.set_options({'basis': 'aug-cc-pvqz',
              'df_basis_scf': 'aug-cc-pvqz-ri',
              'df_basis_mp2': 'aug-cc-pvqz-ri'})
(hf_aqz_cp_l,mp2_aqz_cp_l) = run_curve_rg2_cp("Ne",distances_l)
(Mhf_aqz_cp_l,Mmp2_aqz_cp_l) = run_curve_rg2_cp("Ne",distances_l,midbond=True)

In [ ]:
psi4.set_options({'scf_type': 'pk',
                  'mp2_type': 'conv',
              'basis': 'aug-cc-pvtz',
              'df_basis_scf': 'aug-cc-pvtz-ri',
              'df_basis_mp2': 'aug-cc-pvtz-ri'})
(chf_atz_cp_l,cmp2_atz_cp_l) = run_curve_rg2_cp("Ne",distances_l)
(cMhf_atz_cp_l,cMmp2_atz_cp_l) = run_curve_rg2_cp("Ne",distances_l,midbond=True)
psi4.set_options({'basis': 'aug-cc-pvqz',
              'df_basis_scf': 'aug-cc-pvqz-ri',
              'df_basis_mp2': 'aug-cc-pvqz-ri'})
(chf_aqz_cp_l,cmp2_aqz_cp_l) = run_curve_rg2_cp("Ne",distances_l)
(cMhf_aqz_cp_l,cMmp2_aqz_cp_l) = run_curve_rg2_cp("Ne",distances_l,midbond=True)
psi4.set_options({'scf_type': 'df',
                  'mp2_type': 'df'})

In [ ]:
psi4.set_options({'basis': 'aug-cc-pvtz',
              'df_basis_scf': 'aug-cc-pvtz-ri',
              'df_basis_mp2': 'aug-cc-pvtz-ri',
              'df_basis_sapt': 'aug-cc-pvtz-ri',})
(disp_atz_cp_l,sapt0_atz_cp_l) = run_curve_rg2_sapt0("Ne",distances_l)
(Mdisp_atz_cp_l,Msapt0_atz_cp_l) = run_curve_rg2_sapt0("Ne",distances_l,midbond=True)
psi4.set_options({'basis': 'aug-cc-pvqz',
              'df_basis_scf': 'aug-cc-pvqz-ri',
              'df_basis_mp2': 'aug-cc-pvqz-ri',
              'df_basis_sapt': 'aug-cc-pvqz-ri',})
(disp_aqz_cp_l,sapt0_aqz_cp_l) = run_curve_rg2_sapt0("Ne",distances_l)
(Mdisp_aqz_cp_l,Msapt0_aqz_cp_l) = run_curve_rg2_sapt0("Ne",distances_l,midbond=True)

In [ ]:
eintmp2_atz_cp_l = (mp2_atz_cp_l[:,0] - mp2_atz_cp_l[:,1] - mp2_atz_cp_l[:,2]) * 627.509
eintmp2_aqz_cp_l = (mp2_aqz_cp_l[:,0] - mp2_aqz_cp_l[:,1] - mp2_aqz_cp_l[:,2]) * 627.509
Meintmp2_atz_cp_l = (Mmp2_atz_cp_l[:,0] - Mmp2_atz_cp_l[:,1] - Mmp2_atz_cp_l[:,2]) * 627.509
Meintmp2_aqz_cp_l = (Mmp2_aqz_cp_l[:,0] - Mmp2_aqz_cp_l[:,1] - Mmp2_aqz_cp_l[:,2]) * 627.509
ceintmp2_atz_cp_l = (cmp2_atz_cp_l[:,0] - cmp2_atz_cp_l[:,1] - cmp2_atz_cp_l[:,2]) * 627.509
ceintmp2_aqz_cp_l = (cmp2_aqz_cp_l[:,0] - cmp2_aqz_cp_l[:,1] - cmp2_aqz_cp_l[:,2]) * 627.509
cMeintmp2_atz_cp_l = (cMmp2_atz_cp_l[:,0] - cMmp2_atz_cp_l[:,1] - cMmp2_atz_cp_l[:,2]) * 627.509
cMeintmp2_aqz_cp_l = (cMmp2_aqz_cp_l[:,0] - cMmp2_aqz_cp_l[:,1] - cMmp2_aqz_cp_l[:,2]) * 627.509

plt.close()
plt.plot(distances_l,eintmp2_atz_cp_l,'r',linestyle='-',label='Ne-Ne DF-MP2/aTZ CP')
plt.plot(distances_l,eintmp2_aqz_cp_l,'g',linestyle='-',label='Ne-Ne DF-MP2/aQZ CP')
plt.plot(distances_l,Meintmp2_atz_cp_l,'c',linestyle='--',label='Ne-Ne DF-MP2/aTZ+(aTZ) CP')
plt.plot(distances_l,Meintmp2_aqz_cp_l,'m',linestyle='--',label='Ne-Ne DF-MP2/aQZ+(aQZ) CP')
plt.plot(distances_l,disp_atz_cp_l,'r+',linestyle=' ',label='Ne-Ne SAPT0 disp/aTZ CP')
plt.plot(distances_l,disp_aqz_cp_l,'go',linestyle=' ',label='Ne-Ne SAPT0 disp/aQZ CP')
plt.plot(distances_l,Mdisp_atz_cp_l,'c^',linestyle=' ',label='Ne-Ne SAPT0 disp/aTZ+(aTZ) CP')
plt.plot(distances_l,Mdisp_aqz_cp_l,'y*',linestyle=' ',label='Ne-Ne SAPT0 disp/aQZ+(aQZ) CP')
plt.legend(loc='lower right')
plt.xlim((20.0,50.0))
plt.ylim((-0.0001,0.0))
plt.show()

In [ ]:
plt.close()
plt.plot(distances_l,ceintmp2_atz_cp_l,'r',linestyle='-',label='Ne-Ne Conv. MP2/aTZ CP')
plt.plot(distances_l,ceintmp2_aqz_cp_l,'g',linestyle='-',label='Ne-Ne Conv. MP2/aQZ CP')
plt.plot(distances_l,cMeintmp2_atz_cp_l,'c',linestyle='--',label='Ne-Ne Conv. MP2/aTZ+(aTZ) CP')
plt.plot(distances_l,cMeintmp2_aqz_cp_l,'m',linestyle='--',label='Ne-Ne Conv. MP2/aQZ+(aQZ) CP')
plt.plot(distances_l,disp_atz_cp_l,'r+',linestyle=' ',label='Ne-Ne SAPT0 disp/aTZ CP')
plt.plot(distances_l,disp_aqz_cp_l,'go',linestyle=' ',label='Ne-Ne SAPT0 disp/aQZ CP')
plt.plot(distances_l,Mdisp_atz_cp_l,'c^',linestyle=' ',label='Ne-Ne SAPT0 disp/aTZ+(aTZ) CP')
plt.plot(distances_l,Mdisp_aqz_cp_l,'y*',linestyle=' ',label='Ne-Ne SAPT0 disp/aQZ+(aQZ) CP')
plt.legend(loc='lower right')
plt.xlim((20.0,50.0))
plt.ylim((-0.00003,0.0))
plt.show()
psi4.set_options({'scf_type': 'df',
                  'mp2_type': 'df'})

# 4. Electron density tails

Let's first compute the HF electron density for a neon atom in different bases. We want to be able to compute and display the electron density $\rho(x,y,z)$ at any point in space. Doing this properly in Psi4 is trickier than it seems, but there is an excellent Psi4NumPy tutorial on DFT grids (authored by Victor Chavez) that does exactly what we need. Therefore, big parts of the code below are adapted from this tutorial:

[https://github.com/psi4/psi4numpy/blob/master/Tutorials/04_Density_Functional_Theory/4a_Grids.ipynb](https://github.com/psi4/psi4numpy/blob/master/Tutorials/04_Density_Functional_Theory/4a_Grids.ipynb)

In [ ]:
geometry = psi4.geometry("""Ne 0.0 0.0 0.0
        symmetry c1
        nocom
        noreorient
        units bohr
        """)
psi4.set_options({'basis': 'cc-pvdz',
              'df_basis_scf': 'aug-cc-pvdz-ri',
              'df_basis_mp2': 'aug-cc-pvdz-ri'})
(e,wfn) = psi4.energy('HF',return_wfn=True)
print(e)
# Don't forget to store the wavefunction and the basis set object. 
basis = wfn.basisset()

In [ ]:
# Define the domain with numpy arrays. Unfortunately, we need a pretty dense grid for compact high-Z basis functions.
npoints = 501
z0 = np.linspace(-5, 5, npoints)
y0 = np.linspace(-5, 5, npoints)
x0 = np.linspace(-5, 5, npoints)

# These are one dimensional arrays. Let's use the function meshgrid to define all points in space. 
X,Y,Z = np.meshgrid(x0, y0, z0, indexing='ij')

# This creates a set of points spanning the volume that is of interested. Nevertheless, Psi4 does not handle grids in this way.
# Instead, the input requires an array arranged as (3, total_npoints). Thus we need to reshape and build into a new array. 
# We store the dimensions of the orginal arrays, since they will help us bring the points back to the shape we're interested in.

shape = (len(x0), len(y0), len(z0)) 
X     = X.reshape((X.shape[0] * X.shape[1] * X.shape[2], 1))
Y     = Y.reshape((Y.shape[0] * Y.shape[1] * Y.shape[2], 1))
Z     = Z.reshape((Z.shape[0] * Z.shape[1] * Z.shape[2], 1))
grid  = np.concatenate((X,Y,Z), axis=1).T

#epsilon    = psi4.core.get_global_option("CUBIC_BASIS_TOLERANCE") # Cutoff for basis
#a nonzero epsilon value generates some interesting artifacts at long range
epsilon    = 0.0
max_points = psi4.core.get_global_option("DFT_BLOCK_MAX_POINTS")  # Maximum number of points per block
npoints    = grid.shape[1]                                        # Total number of points
extens     = psi4.core.BasisExtents(basis, epsilon)
nblocks    = int(np.floor(npoints/max_points))
remainder  = npoints - (max_points * nblocks)
blocks     = []

max_functions = 0

# Run through blocks
idx = 0 
inb = 0
for nb in range(nblocks+1):
    x = psi4.core.Vector.from_array(grid[0][idx : idx + max_points if inb < nblocks else idx + remainder])
    y = psi4.core.Vector.from_array(grid[1][idx : idx + max_points if inb < nblocks else idx + remainder])
    z = psi4.core.Vector.from_array(grid[2][idx : idx + max_points if inb < nblocks else idx + remainder])
    w = psi4.core.Vector.from_array(np.zeros(max_points))

    blocks.append( psi4.core.BlockOPoints(x, y, z, w, extens) )
    idx += max_points if inb < nblocks else remainder
    inb += 1
    max_functions = max_functions if max_functions > len(blocks[-1].functions_local_to_global()) \
                                                     else len(blocks[-1].functions_local_to_global())
    
    
points_func = psi4.core.RKSFunctions(basis, max_points, max_functions)
points_func.set_ansatz(0)

In [ ]:
# Lets create a dictionary to store all of our orbitals
orbitals_p = np.zeros( (basis.nbf(), npoints) ) 

# And proceed just like we did with the LDA kernet tutorial. 
points_func.set_pointers( wfn.Da() )
Ca_np = np.array( wfn.Ca() )
#Ca_np = np.identity(basis.nbf())

offset = 0
for i_block in blocks:
    points_func.compute_points(i_block)
    b_points = i_block.npoints()
    offset += b_points
    lpos = np.array( i_block.functions_local_to_global() )
    
    if len(lpos) == 0:
        continue

    # Extract the subset of the Phi matrix. 
    phi = np.array(points_func.basis_values()["PHI"])[:b_points, :lpos.shape[0]]
    # Extract the subset of Ca as well
    Ca_local = Ca_np[(lpos[:, None], lpos)]
    # Store in the correct spot
    orbitals_p[lpos, offset - b_points : offset] = Ca_local.T @ phi.T
    
#How many points did we started with?
print("Total points in mesh", shape[0]*shape[1]*shape[2])

#How many points are in it?
print("Total points in the orbital", orbitals_p.shape)

print(orbitals_p[0,:])
print(np.max(orbitals_p[0,:]),np.argmax(orbitals_p[0,:]))
print(orbitals_p[:,62875750])

Values of s functions at origin are correct, also for contractions (checked it).

Let's now use the 5 occupied orbitals to build the neon atom density. For the first test, we'll integrate the density over all space.

In [ ]:
# In order to compute (2), we carry a simple numerical integration. For our current grid, the differential is simply our spacing cubed.  
diff = (x0[1]-x0[0])**3
noccorb = 5
rho  = np.abs( orbitals_p[:noccorb,:] ) ** 2

# And we can simply use the np.sum function to find the value of our integral. 
neach = np.array([np.sum(rho[i,:] * diff ) for i in range(noccorb)])
N = 2.0 * np.sum(neach)  # 2 is for spin-up and spin-down electrons

#print(neach)
print(f"Our computed density has: {N} electrons")


Now that we are confident that the code is working, let's prepare numbers for a couple basis sets.

In [ ]:
bases_all = ['cc-pvdz','aug-cc-pvdz','cc-pvtz','aug-cc-pvtz','cc-pvqz','aug-cc-pvqz']
basesri_all = ['aug-cc-pvdz-ri','aug-cc-pvdz-ri','aug-cc-pvtz-ri','aug-cc-pvtz-ri','aug-cc-pvqz-ri','aug-cc-pvqz-ri']
npoints = 2001
z2 = np.linspace(-20, 20, npoints)

def compute_all_densities(method,bases,basesri,noccorb):
    allrhos = []
    x1 = np.array([0.0])
    y1 = np.array([0.0])    
    X,Y,Z = np.meshgrid(x1, y1, z2, indexing='ij')

    shape = (len(x1), len(y1), len(z2)) 
    X     = X.reshape((X.shape[0] * X.shape[1] * X.shape[2], 1))
    Y     = Y.reshape((Y.shape[0] * Y.shape[1] * Y.shape[2], 1))
    Z     = Z.reshape((Z.shape[0] * Z.shape[1] * Z.shape[2], 1))
    smallgrid  = np.concatenate((X,Y,Z), axis=1).T

    for (bas,basri) in zip(bases,basesri):
        psi4.core.clean()
        psi4.set_options({'basis': bas,
                  'df_basis_scf': basri,
                  'df_basis_mp2': basri})

        (e,wfn) = psi4.energy(method,return_wfn=True)
        print(bas,e)
        basis = wfn.basisset()
    
        npoints    = smallgrid.shape[1]                                        # Total number of points
        extens     = psi4.core.BasisExtents(basis, epsilon)
        nblocks    = int(np.floor(npoints/max_points))
        remainder  = npoints - (max_points * nblocks)
        blocks     = []
        max_functions = 0

        # Run through blocks
        idx = 0 
        inb = 0
        for nb in range(nblocks+1):
            x = psi4.core.Vector.from_array(smallgrid[0][idx : idx + max_points if inb < nblocks else idx + remainder])
            y = psi4.core.Vector.from_array(smallgrid[1][idx : idx + max_points if inb < nblocks else idx + remainder])
            z = psi4.core.Vector.from_array(smallgrid[2][idx : idx + max_points if inb < nblocks else idx + remainder])
            w = psi4.core.Vector.from_array(np.zeros(max_points))

            blocks.append( psi4.core.BlockOPoints(x, y, z, w, extens) )
            idx += max_points if inb < nblocks else remainder
            inb += 1
            max_functions = max_functions if max_functions > len(blocks[-1].functions_local_to_global()) \
                                                         else len(blocks[-1].functions_local_to_global())
       
        points_func = psi4.core.RKSFunctions(basis, max_points, max_functions)
        points_func.set_ansatz(0)
    
        orbitals_p = np.zeros( (basis.nbf(), npoints) ) 
        points_func.set_pointers( wfn.Da() )
        Ca_np = np.array( wfn.Ca() )

        offset = 0
        for i_block in blocks:
            points_func.compute_points(i_block)
            b_points = i_block.npoints()
            offset += b_points
            lpos = np.array( i_block.functions_local_to_global() )    
            if len(lpos) == 0:
                continue 
            phi = np.array(points_func.basis_values()["PHI"])[:b_points, :lpos.shape[0]]   
            Ca_local = Ca_np[(lpos[:, None], lpos)]
            orbitals_p[lpos, offset - b_points : offset] = Ca_local.T @ phi.T
    
        rho  = np.power(orbitals_p[:noccorb,:], 2.0) 
        rhoall = 2.0 * np.sum(rho, axis=0)  # 2.0 factor is for spins alpha and beta
        allrhos.append(rhoall)

    allrhos = np.array(allrhos)
    return allrhos

allrhos = compute_all_densities('HF',bases_all,basesri_all,5)

In [ ]:
plt.close()
plt.plot(z2,allrhos[0,:],'g',linestyle='-',label='cc-pvdz')
plt.plot(z2,allrhos[1,:],'g',linestyle='--',label='aug-cc-pvdz')
plt.plot(z2,allrhos[2,:],'b',linestyle='-',label='cc-pvtz')
plt.plot(z2,allrhos[3,:],'b',linestyle='--',label='aug-cc-pvtz')
plt.plot(z2,allrhos[4,:],'c',linestyle='-',label='cc-pvqz')
plt.plot(z2,allrhos[5,:],'c',linestyle='--',label='aug-cc-pvqz')
plt.legend(loc='upper right')
plt.xlim((-5.0,5.0))
plt.ylim((0.0,10.0))
plt.show()

At this scale the densities seem identical, so we need to zoom in on the long range (the **tail**):

In [ ]:
plt.close()
plt.plot(z2,allrhos[0,:],'g',linestyle='-',label='cc-pvdz')
plt.plot(z2,allrhos[1,:],'g',linestyle='--',label='aug-cc-pvdz')
plt.plot(z2,allrhos[2,:],'b',linestyle='-',label='cc-pvtz')
plt.plot(z2,allrhos[3,:],'b',linestyle='--',label='aug-cc-pvtz')
plt.plot(z2,allrhos[4,:],'c',linestyle='-',label='cc-pvqz')
plt.plot(z2,allrhos[5,:],'c',linestyle='--',label='aug-cc-pvqz')
plt.legend(loc='upper right')
plt.xlim((3.0,5.0))
plt.ylim((0.0,0.0005))
plt.show()

In [ ]:
plt.close()
plt.plot(z2,allrhos[0,:],'g',linestyle='-',label='cc-pvdz')
plt.plot(z2,allrhos[1,:],'g',linestyle='--',label='aug-cc-pvdz')
plt.plot(z2,allrhos[2,:],'b',linestyle='-',label='cc-pvtz')
plt.plot(z2,allrhos[3,:],'b',linestyle='--',label='aug-cc-pvtz')
plt.plot(z2,allrhos[4,:],'c',linestyle='-',label='cc-pvqz')
plt.plot(z2,allrhos[5,:],'c',linestyle='--',label='aug-cc-pvqz')
plt.legend(loc='upper right')
plt.xlim((3.5,8.0))
plt.ylim((0.0,0.00001))
plt.show()

Let's now focus on the $[3.0,6.0)$ range and fit the density tails with exponential and Gaussian functions. Let's do that specifically for the aug-cc-pVTZ basis, which was number 4 on our list (therefore, that density is stored in `allrhos[3,:]`).

In [ ]:
# we will fit expressions in the [3.0,6.0) range and need to know the indices in the z2 array
rstart = 1150
rend = 1300
print(z2[rstart])
print(z2[rend])

For an atom, the exact form of the long-range decay of electron density is known:
$$ \rho(r) \propto r^{2\beta} e^{-2\alpha r} $$
where $\alpha$ is related to the ionization potential $I$ as $\alpha=\sqrt{2I}$, and $\beta=-1+1/\alpha$. Let's compare our atomic densities to their exact long-range form.

In [ ]:
def exponential(r,param1,param2):
#We need to fit a function rho(R) = a*exp(-bR)
  return param1*np.exp(-param2 * r)

def gaussian(r,param1,param2):
#We need to fit a function rho(R) = a*exp(-bR^2)
  return param1*np.exp(-param2 * r * r)

parexp = np.zeros(6)
pargau = np.zeros(6)
conexp = np.zeros(6)
congau = np.zeros(6)
for i in range(6):
    (conexp[i],parexp[i]) = scipy.optimize.curve_fit(exponential,z2[rstart:rend],allrhos[i,rstart:rend])[0][:2]
    (congau[i],pargau[i]) = scipy.optimize.curve_fit(gaussian,z2[rstart:rend],allrhos[i,rstart:rend])[0][:2]
    
print('Optimal Slater exponents for density tails:',parexp)
print('Optimal Gaussian exponents for density tails:',pargau)
# save Slater aug-cc-pVTZ values for later
D_Slater = conexp[3]
B_Slater = parexp[3]

#now the true decay of atomic electron density (Van Vleet et al. JCTC 2016, 12, 3851 eq. (3))
def true_asymp(r):
    Ip_Ne_in_au = 0.79248  # NIST Webbook
    alpha = np.sqrt(2.0 * Ip_Ne_in_au)
    beta = -1.0 + 1.0/alpha
    return np.exp(-2.0 * alpha * r) * np.power(r, 2.0 * beta)

expval = []
gauval = []
trueasympval = []
for i in range(rstart,rend):
    expval.append(exponential(z2[i],conexp[3],parexp[3]))
    gauval.append(gaussian(z2[i],congau[3],pargau[3]))
    trueasympval.append(true_asymp(z2[i]))
expval = np.array(expval)
gauval = np.array(gauval)
trueasympval = np.array(trueasympval)


plt.close()
plt.plot(z2[rstart:rend],allrhos[3,rstart:rend],'b',linestyle='--',label='aug-cc-pvtz')
plt.plot(z2[rstart:rend],expval,'g',linestyle='-',label='exponential fit')
plt.plot(z2[rstart:rend],gauval,'c',linestyle='-',label='Gaussian fit')
plt.plot(z2[rstart:rend],trueasympval,'k',linestyle=':',label='exact asymptotics')
plt.legend(loc='upper right')
plt.xlim((4.0,6.0))
plt.ylim((0.0,0.00002))
plt.title('HF/aug-cc-pVTZ')
plt.show()

Beautiful! Let's do the same for some DFT starting with nonhybrid PBE, followed by hybrid PBE0 and range-separated hybrid LC-$\omega$PBE.

In [ ]:
allrhos_pbe = compute_all_densities('PBE',bases_all,basesri_all,5)

parexp = np.zeros(6)
pargau = np.zeros(6)
conexp = np.zeros(6)
congau = np.zeros(6)
for i in range(6):
    (conexp[i],parexp[i]) = scipy.optimize.curve_fit(exponential,z2[rstart:rend],allrhos_pbe[i,rstart:rend])[0][:2]
    (congau[i],pargau[i]) = scipy.optimize.curve_fit(gaussian,z2[rstart:rend],allrhos_pbe[i,rstart:rend])[0][:2]
    
print('Optimal Slater exponents for density tails:',parexp)
print('Optimal Gaussian exponents for density tails:',pargau)

expval = []
gauval = []
for i in range(rstart,rend):
    expval.append(exponential(z2[i],conexp[3],parexp[3]))
    gauval.append(gaussian(z2[i],congau[3],pargau[3]))
expval = np.array(expval)
gauval = np.array(gauval)

plt.close()
plt.plot(z2[rstart:rend],allrhos_pbe[3,rstart:rend],'b',linestyle='--',label='aug-cc-pvtz')
plt.plot(z2[rstart:rend],expval,'g',linestyle='-',label='exponential fit')
plt.plot(z2[rstart:rend],gauval,'c',linestyle='-',label='Gaussian fit')
plt.plot(z2[rstart:rend],trueasympval,'k',linestyle=':',label='exact asymptotics')
plt.legend(loc='upper right')
plt.xlim((4.0,6.0))
plt.ylim((0.0,0.00002))
plt.title('PBE/aug-cc-pVTZ')
plt.show()

In [ ]:
allrhos_pbe0 = compute_all_densities('PBE0',bases_all,basesri_all,5)

parexp = np.zeros(6)
pargau = np.zeros(6)
conexp = np.zeros(6)
congau = np.zeros(6)
for i in range(6):
    (conexp[i],parexp[i]) = scipy.optimize.curve_fit(exponential,z2[rstart:rend],allrhos_pbe0[i,rstart:rend])[0][:2]
    (congau[i],pargau[i]) = scipy.optimize.curve_fit(gaussian,z2[rstart:rend],allrhos_pbe0[i,rstart:rend])[0][:2]
    
print('Optimal Slater exponents for density tails:',parexp)
print('Optimal Gaussian exponents for density tails:',pargau)

expval = []
gauval = []
for i in range(rstart,rend):
    expval.append(exponential(z2[i],conexp[3],parexp[3]))
    gauval.append(gaussian(z2[i],congau[3],pargau[3]))
expval = np.array(expval)
gauval = np.array(gauval)

plt.close()
plt.plot(z2[rstart:rend],allrhos_pbe0[3,rstart:rend],'b',linestyle='--',label='aug-cc-pvtz')
plt.plot(z2[rstart:rend],expval,'g',linestyle='-',label='exponential fit')
plt.plot(z2[rstart:rend],gauval,'c',linestyle='-',label='Gaussian fit')
plt.plot(z2[rstart:rend],trueasympval,'k',linestyle=':',label='exact asymptotics')
plt.legend(loc='upper right')
plt.xlim((4.0,6.0))
plt.ylim((0.0,0.00002))
plt.title('PBE0/aug-cc-pVTZ')
plt.show()

In [ ]:
psi4.set_options({'dft_omega': 0.4}) #default for LC-omegaPBE
allrhos_lcwpbe = compute_all_densities('LC-WPBE',bases_all,basesri_all,5)

parexp = np.zeros(6)
pargau = np.zeros(6)
conexp = np.zeros(6)
congau = np.zeros(6)
for i in range(6):
    (conexp[i],parexp[i]) = scipy.optimize.curve_fit(exponential,z2[rstart:rend],allrhos_lcwpbe[i,rstart:rend])[0][:2]
    (congau[i],pargau[i]) = scipy.optimize.curve_fit(gaussian,z2[rstart:rend],allrhos_lcwpbe[i,rstart:rend])[0][:2]
    
print('Optimal Slater exponents for density tails:',parexp)
print('Optimal Gaussian exponents for density tails:',pargau)

expval = []
gauval = []
for i in range(rstart,rend):
    expval.append(exponential(z2[i],conexp[3],parexp[3]))
    gauval.append(gaussian(z2[i],congau[3],pargau[3]))
expval = np.array(expval)
gauval = np.array(gauval)

plt.close()
plt.plot(z2[rstart:rend],allrhos_lcwpbe[3,rstart:rend],'b',linestyle='--',label='aug-cc-pvtz')
plt.plot(z2[rstart:rend],expval,'g',linestyle='-',label='exponential fit')
plt.plot(z2[rstart:rend],gauval,'c',linestyle='-',label='Gaussian fit')
plt.plot(z2[rstart:rend],trueasympval,'k',linestyle=':',label='exact asymptotics')
plt.legend(loc='upper right')
plt.xlim((4.0,6.0))
plt.ylim((0.0,0.00002))
plt.title('LC-$\omega$PBE/aug-cc-pVTZ')
plt.show()

The reason LC-$\omega$PBE did not do so well is because the default $\omega$ value is inappropriate four our purpose. Let's pick a *tuned* value of $\omega$ from the global density-dependent (GDD) algorithm:

In [ ]:
psi4.set_options({'dft_omega': 0.697}) # GDD value computed with Q-Chem
allrhos_tlcwpbe = compute_all_densities('LC-WPBE',bases_all,basesri_all,5)

parexp = np.zeros(6)
pargau = np.zeros(6)
conexp = np.zeros(6)
congau = np.zeros(6)
for i in range(6):
    (conexp[i],parexp[i]) = scipy.optimize.curve_fit(exponential,z2[rstart:rend],allrhos_tlcwpbe[i,rstart:rend])[0][:2]
    (congau[i],pargau[i]) = scipy.optimize.curve_fit(gaussian,z2[rstart:rend],allrhos_tlcwpbe[i,rstart:rend])[0][:2]
    
print('Optimal Slater exponents for density tails:',parexp)
print('Optimal Gaussian exponents for density tails:',pargau)

expval = []
gauval = []
for i in range(rstart,rend):
    expval.append(exponential(z2[i],conexp[3],parexp[3]))
    gauval.append(gaussian(z2[i],congau[3],pargau[3]))
expval = np.array(expval)
gauval = np.array(gauval)

plt.close()
plt.plot(z2[rstart:rend],allrhos_tlcwpbe[3,rstart:rend],'b',linestyle='--',label='aug-cc-pvtz')
plt.plot(z2[rstart:rend],expval,'g',linestyle='-',label='exponential fit')
plt.plot(z2[rstart:rend],gauval,'c',linestyle='-',label='Gaussian fit')
plt.plot(z2[rstart:rend],trueasympval,'k',linestyle=':',label='exact asymptotics')
plt.legend(loc='upper right')
plt.xlim((4.0,6.0))
plt.ylim((0.0,0.00002))
plt.title('GDD-Tuned LC-$\omega$PBE/aug-cc-pVTZ')
plt.show()

Instead of tuning the range-separated hybrid functional, we can pick a regular hybrid like PBE0 and *asymptotically correct* it with the GRAC scheme, just like for SAPT(DFT).

In [ ]:
#Find the GRAC shift for neon
(e,wfn) = psi4.energy('PBE0',return_wfn=True)
orbital_energies = np.array(wfn.epsilon_a())
eps_homo_pbe0_ne = orbital_energies[4]
print(orbital_energies)
print(eps_homo_pbe0_ne)

In [ ]:
Ip_Ne_in_au = 0.79248  # NIST Webbook
shift_ne=Ip_Ne_in_au+eps_homo_pbe0_ne
psi4.set_options({'dft_grac_shift': shift_ne})
allrhos_gracpbe0 = compute_all_densities('PBE0',bases_all,basesri_all,5)

parexp = np.zeros(6)
pargau = np.zeros(6)
conexp = np.zeros(6)
congau = np.zeros(6)
for i in range(6):
    (conexp[i],parexp[i]) = scipy.optimize.curve_fit(exponential,z2[rstart:rend],allrhos_gracpbe0[i,rstart:rend])[0][:2]
    (congau[i],pargau[i]) = scipy.optimize.curve_fit(gaussian,z2[rstart:rend],allrhos_gracpbe0[i,rstart:rend])[0][:2]
    
print('Optimal Slater exponents for density tails:',parexp)
print('Optimal Gaussian exponents for density tails:',pargau)

expval = []
gauval = []
for i in range(rstart,rend):
    expval.append(exponential(z2[i],conexp[3],parexp[3]))
    gauval.append(gaussian(z2[i],congau[3],pargau[3]))
expval = np.array(expval)
gauval = np.array(gauval)

plt.close()
plt.plot(z2[rstart:rend],allrhos_gracpbe0[3,rstart:rend],'b',linestyle='--',label='aug-cc-pvtz')
plt.plot(z2[rstart:rend],expval,'g',linestyle='-',label='exponential fit')
plt.plot(z2[rstart:rend],gauval,'c',linestyle='-',label='Gaussian fit')
plt.plot(z2[rstart:rend],trueasympval,'k',linestyle=':',label='exact asymptotics')
plt.legend(loc='upper right')
plt.xlim((4.0,6.0))
plt.ylim((0.0,0.00002))
plt.title('GRAC PBE0/aug-cc-pVTZ')
plt.show()

psi4.set_options({'dft_grac_shift': 0.0})

In [ ]:
plt.close()
plt.plot(z2[rstart:rend],allrhos[3,rstart:rend],'m',linestyle='-',label='HF/aug-cc-pVTZ')
plt.plot(z2[rstart:rend],allrhos_pbe[3,rstart:rend],'g',linestyle='-',label='PBE/aug-cc-pVTZ')
plt.plot(z2[rstart:rend],allrhos_pbe0[3,rstart:rend],'c',linestyle='-',label='PBE0/aug-cc-pVTZ')
plt.plot(z2[rstart:rend],allrhos_lcwpbe[3,rstart:rend],'r',linestyle='-',label='LC-$\omega$PBE/aug-cc-pVTZ')
plt.plot(z2[rstart:rend],allrhos_tlcwpbe[3,rstart:rend],'b',linestyle='-',label='GDD-LC-$\omega$PBE/aug-cc-pVTZ')
plt.plot(z2[rstart:rend],allrhos_gracpbe0[3,rstart:rend],'o',linestyle=' ',label='GRAC PBE0/aug-cc-pVTZ')
plt.plot(z2[rstart:rend],trueasympval,'k',linestyle=':',label='exact asymptotics')
plt.legend(loc='upper right')
plt.xlim((4.0,5.5))
plt.ylim((0.0,0.00002))
plt.title('Ne density tail comparison')
plt.show()

# Density overlap

The minimum distance in the neon dimer is 3.1 Angstrom, so about 5.9 bohr. We will place two neon atoms at (0,0,0) and (0,0,5.9) and model the overlap of the two HF/aug-cc-pVTZ densities. To be able to compute the density at any point in space, we will interpolate the radial values we have computed before.

In [ ]:
def interpolated_density(r):
#to preserve exponential character, we linearly interpolate the natural logarithm of density
    den = allrhos[3,1000:]  # 1000: corresponds to positive R
    znum = int(r * 50.0)
    if znum >= len(den)-1:
        return 0.0
    lnrho0 = np.log(den[znum]+1.e-20)
    lnrho1 = np.log(den[znum+1]+1.e-20)
    lnrhoint = lnrho0 +(lnrho1 - lnrho0)*(r-z2[znum+1000])/(z2[znum+1001]-z2[znum+1000])
    rhoint = np.exp(lnrhoint)
    return rhoint

def atomic_density_product(xyz,z2):
    nuc1 = np.array([0.0,0.0,0.0])
    nuc2 = np.array([0.0,0.0,z2])
    r1 = np.sqrt(np.sum(np.power(xyz - nuc1,2.0)))
    r2 = np.sqrt(np.sum(np.power(xyz - nuc2,2.0)))
    return interpolated_density(r1) * interpolated_density(r2)

# Let's do the xz cross-section
xrange = np.linspace(-5.0,5.0,501)
zrange = np.linspace(-5.0,10.9,796)
X,Z = np.meshgrid(xrange, zrange, indexing='ij')

denovl = np.zeros_like(X)
for i in range(len(X)):
    xrow = X[i]
    zrow = Z[i]
    for j in range(len(xrow)):
        x = xrow[j]
        z = zrow[j]
        denovl[i,j] = atomic_density_product(np.array([x,0.0,z]),5.9)


In [ ]:
from matplotlib import ticker
print(np.max(denovl),np.argmax(denovl))
plt.close()
plt.contourf(X, Z, denovl, locator=ticker.LogLocator(), cmap="summer")
plt.colorbar()
plt.show()

Now we would like to integrate that density overlap. If we assume cylindrical symmetry, we should be able to integrate from a 2D cross-section.

$$ \int\int\int f(x,y,z)\, dxdydz = \int\int\int f(r,z)\, rdrd\phi dz = 2\pi \int\int rf(r,z)\, drdz $$

In [ ]:
def integrate_overlap(denovl, X, Z):
    xincr = 0.02
    yincr = 0.02
    int_ovl = 0.0

    for i in range(len(X)):
        xrow = X[i]
        zrow = Z[i]
        for j in range(len(xrow)):
            x = xrow[j]
            z = zrow[j]
            if x < 0:
                continue
            int_ovl += denovl[i,j] * x * xincr * yincr

    int_ovl *= 2.0 * np.pi
    return int_ovl

print('Density overlap integral:',integrate_overlap(denovl, X, Z))

Now we are ready to explore how the density overlap integral depends on the atom-atom separation.

In [ ]:
dist_nene = [5.0,5.3,5.6,5.9,6.2,6.5,7.0,7.5,8.0,8.5,9.0]
ov_nene = []
for r in dist_nene:
    xrange = np.linspace(-5.0,5.0,501)
    zrange = np.linspace(-5.0,5.0 + r,int(501 + r * 50))
    X,Z = np.meshgrid(xrange, zrange, indexing='ij')
    denov = np.zeros_like(X)
    for i in range(len(X)):
        xrow = X[i]
        zrow = Z[i]
        for j in range(len(xrow)):
            x = xrow[j]
            z = zrow[j]
            denov[i,j] = atomic_density_product(np.array([x,0.0,z]),r)
    ov = integrate_overlap(denov, X, Z)
    print(r,ov)
    ov_nene.append(ov)

In [ ]:
#eq. (5) of Van Vleet et al. 2016, based on the exponential density decay
def density_overlap_Slater(r):
    Br = B_Slater * r
    poly = Br * Br / 3.0 + Br + 1.0
    con = np.pi * np.power(D_Slater, 2.0) / np.power(B_Slater, 3.0)
    return con * poly * np.exp(-Br)

rov = np.linspace(5.0, 9.0, 41)
dov = []
for r in rov:
    dov.append(density_overlap_Slater(r))

plt.close()
plt.plot(dist_nene,ov_nene,'bo',linestyle=' ',label='HF/aug-cc-pVTZ')
plt.plot(rov,dov,'g',linestyle='-',label='exp. overlap model')
plt.legend(loc='upper right')
plt.xlim((5.0,9.0))
plt.ylim((0.0,0.000012))
plt.title('Overlap of two Ne atom densities')
plt.show()


To take our visualization of density overlap one step further, we will now generate this function on an $(x,y,z)$ grid, writing the results on disk in the standard *cube file* format. This format can be read by many different programs, and we can use Francesco Evangelista's `fortecubeview` library to visualize it directly in the notebook.

In [ ]:
def generate_density_overlap_cube(r,spacing,scaling):
    npoints_x = int(10.0 / spacing) + 1
    npoints_z = int((10.0 + r)/ spacing) + 1
    xcube = np.linspace(-5.0, 5.0, npoints_x)
    zcube = np.linspace(-5.0, 5.0 + r, npoints_z)

    lcube = []
    lcube.append("Density overlap cube file")
    lcube.append("Two neon atoms separated by "+str(r)+" bohr")
    lcube.append("      2  -5.000000  -5.000000  -5.000000")
    snx = "%7d" % npoints_x
    snz = "%7d" % npoints_z
    ssp = "%8.6f" % spacing
#     1  -5.000000  -5.000000  -5.000000
#    51   0.200000   0.000000   0.000000
#    51   0.000000   0.200000   0.000000
#    51   0.000000   0.000000   0.200000
# 10   0.000000   0.000000   0.000000   0.000000   
    lcube.append(snx+"   "+ssp+"   0.000000   0.000000")
    lcube.append(snx+"   0.000000   "+ssp+"   0.000000")
    lcube.append(snz+"   0.000000   0.000000   "+ssp)
    lcube.append(" 10   0.000000   0.000000   0.000000   0.000000")
    lcube.append(" 10   0.000000   0.000000   0.000000  "+("%9.6f" % r))
    lval = []
    for xpoint in xcube:
        for ypoint in xcube:
            for zpoint in zcube:
                denovpoint = atomic_density_product(np.array([xpoint,ypoint,zpoint]),r)
                sdenov = "%11.5e" % (denovpoint * scaling)
                lval.append(sdenov)
                if len(lval) == 6:
                    lcube.append(" "+"  ".join(lval))
                    lval = []
    if len(lval) > 0:
        lcube.append(" "+"  ".join(lval))
    f = open("Ddenov"+str(r).strip()+".cube","w")
    for s in lcube:
        f.write(s+"\n")
    f.close()
    return 1

    
dummy = generate_density_overlap_cube(5.9,0.1,100000.0)

In [ ]:
import fortecubeview
fortecubeview.plot(width=500,height=300,colorscheme="emory",sumlevel=0.97)

# SAPT first-order exchange versus density overlap

There are strong theoretical arguments that the SAPT first-order exchange energy $E^{(1)}_{\rm exch}$ between two molecules is (nearly) proportional to the overlap of their electron densities. Let's check if we see such a proportionality in our case.

In [ ]:
psi4.set_options({'basis': 'aug-cc-pvtz',
              'df_basis_scf': 'aug-cc-pvtz-ri',
              'df_basis_sapt': 'aug-cc-pvtz-ri',
              'df_basis_mp2': 'aug-cc-pvtz-ri'})
eelst = np.zeros((len(dist_nene)))
eexch = np.zeros((len(dist_nene)))
eind = np.zeros((len(dist_nene)))
edisp = np.zeros((len(dist_nene)))
esapt = np.zeros((len(dist_nene)))

for i in range(len(dist_nene)):
  dimer = psi4.geometry("""
  Ne 0.0 0.0 0.0
  --
  Ne 0.0 0.0 """+str(dist_nene[i])+"""
  units bohr
  symmetry c1
  """)

  psi4.energy('sapt0')
  eelst[i] = psi4.variable('SAPT ELST ENERGY') * 627.509
  eexch[i] = psi4.variable('SAPT EXCH ENERGY') * 627.509
  eind[i] = psi4.variable('SAPT IND ENERGY') * 627.509
  edisp[i] = psi4.variable('SAPT DISP ENERGY') * 627.509
  esapt[i] = psi4.variable('SAPT TOTAL ENERGY') * 627.509
  psi4.core.clean()

plt.close()
plt.ylim(-0.2,0.4)
plt.plot(dist_nene,eelst,'r+',linestyle='-',label='SAPT0 elst')
plt.plot(dist_nene,eexch,'bo',linestyle='-',label='SAPT0 exch')
plt.plot(dist_nene,eind,'g^',linestyle='-',label='SAPT0 ind')
plt.plot(dist_nene,edisp,'mx',linestyle='-',label='SAPT0 disp')
plt.plot(dist_nene,esapt,'k*',linestyle='-',label='SAPT0 total')
plt.legend(loc='upper right')
plt.show()

That was SAPT0, now let's do the same thing in SAPT(DFT). We will do it both correctly (with the proper GRAC asymptotic correction) and badly (without the asymptotic correction).

In [ ]:
#for SAPT(DFT), the GRAC shifts have to be specified
psi4.set_options({'sapt_dft_grac_shift_A': shift_ne,
                  'sapt_dft_grac_shift_B': shift_ne,
                  'sapt_dft_mp2_disp_alg': 'fisapt'})

eelst_saptdft = np.zeros((len(dist_nene)))
eexch_saptdft = np.zeros((len(dist_nene)))
eind_saptdft = np.zeros((len(dist_nene)))
edisp_saptdft = np.zeros((len(dist_nene)))
esapt_saptdft = np.zeros((len(dist_nene)))

for i in range(len(dist_nene)):
  dimer = psi4.geometry("""
  Ne 0.0 0.0 0.0
  --
  Ne 0.0 0.0 """+str(dist_nene[i])+"""
  units bohr
  symmetry c1
  """)
    
  psi4.energy('sapt(dft)')  # PBE0 is used by default
  eelst_saptdft[i] = psi4.variable('SAPT ELST ENERGY') * 627.509
  eexch_saptdft[i] = psi4.variable('SAPT EXCH ENERGY') * 627.509
  eind_saptdft[i] = psi4.variable('SAPT IND ENERGY') * 627.509
  edisp_saptdft[i] = psi4.variable('SAPT DISP ENERGY') * 627.509
  esapt_saptdft[i] = psi4.variable('SAPT TOTAL ENERGY') * 627.509
  print('Completed SAPT(DFT) for point',i+1)
  psi4.core.clean()


psi4.set_options({'sapt_dft_grac_shift_A': 0.00001,
                  'sapt_dft_grac_shift_B': 0.00001}) # values of exactly 0.0 would cause error

# now let's run BAD SAPT(DFT) without asymptotic correction
bad_eelst_saptdft = np.zeros((len(dist_nene)))
bad_eexch_saptdft = np.zeros((len(dist_nene)))
bad_eind_saptdft = np.zeros((len(dist_nene)))
bad_edisp_saptdft = np.zeros((len(dist_nene)))
bad_esapt_saptdft = np.zeros((len(dist_nene)))

for i in range(len(dist_nene)):
  dimer = psi4.geometry("""
  Ne 0.0 0.0 0.0
  --
  Ne 0.0 0.0 """+str(dist_nene[i])+"""
  units bohr
  symmetry c1
  """)
    
  psi4.energy('sapt(dft)')  # PBE0 is used by default
  bad_eelst_saptdft[i] = psi4.variable('SAPT ELST ENERGY') * 627.509
  bad_eexch_saptdft[i] = psi4.variable('SAPT EXCH ENERGY') * 627.509
  bad_eind_saptdft[i] = psi4.variable('SAPT IND ENERGY') * 627.509
  bad_edisp_saptdft[i] = psi4.variable('SAPT DISP ENERGY') * 627.509
  bad_esapt_saptdft[i] = psi4.variable('SAPT TOTAL ENERGY') * 627.509
  print('Completed BAD SAPT(DFT) for point',i+1)
  psi4.core.clean()


plt.close()
plt.xlim(5.0,9.0)
plt.plot(dist_nene,eelst_saptdft,'r',linestyle='-',label='SAPT(DFT) elst')
plt.plot(dist_nene,bad_eelst_saptdft,'r+',linestyle=' ',label='Bad SAPT(DFT) elst')
plt.plot(dist_nene,eexch_saptdft,'b',linestyle='-',label='SAPT(DFT) exch')
plt.plot(dist_nene,bad_eexch_saptdft,'bo',linestyle=' ',label='Bad SAPT(DFT) exch')
plt.plot(dist_nene,eind_saptdft,'g',linestyle='-',label='SAPT(DFT) ind')
plt.plot(dist_nene,bad_eind_saptdft,'g^',linestyle=' ',label='Bad SAPT(DFT) ind')
plt.plot(dist_nene,edisp_saptdft,'m',linestyle='-',label='SAPT(DFT) disp')
plt.plot(dist_nene,bad_edisp_saptdft,'mx',linestyle=' ',label='Bad SAPT(DFT) disp')
plt.plot(dist_nene,esapt_saptdft,'k',linestyle='-',label='SAPT(DFT) total')
plt.plot(dist_nene,bad_esapt_saptdft,'k*',linestyle=' ',label='Bad SAPT(DFT) total')
plt.legend(loc='upper right')
plt.show()

psi4.set_options({'reference':'rhf'}) # somehow SAPT(DFT) resets the reference

In [ ]:
def ax_fun(x,param):
    return param * x

slope = scipy.optimize.curve_fit(ax_fun,ov_nene,eexch_saptdft)[0][0]
print('Slope of the line of best fit:',slope)
rsl = np.linspace(0.0,0.00014,141)

plt.close()
plt.scatter(ov_nene,eexch_saptdft)
plt.plot(rsl,slope*rsl,'g',linestyle='-')
plt.xlabel('Atomic density overlap [a.u.]')
plt.ylabel('SAPT(DFT) first-order exchange [kcal/mol]')
plt.show()

# HF-HF interaction energy at long distances

For the next exploration, it's most convenient to pick two molecules that are polar (so they exhibit a dipole-dipole interaction, decaying with distance as $R^{-3}$) but linear (so that the dipole-quadrupole and quadrupole-quadrupole terms are not super complicated). Therefore, let's pick the hydrogen fluoride dimer, starting from the minimum geometry taken from the A24 database. Since we want to play with the intermolecular distance and angles, we want to change the Cartesian geometry into a Z-matrix.

In [ ]:
#HF-HF geometry from the A24 database
H1 = np.array([   0.000000000,   0.802679820,   1.695293290])
F1 = np.array([   0.000000000,  -0.045966660,   1.340348180])
H2 = np.array([   0.000000000,  -0.120407870,  -0.490828400])
F2 = np.array([   0.000000000,   0.009769450,  -1.404249780])
mH = 1.0078
mF = 18.9984
com1 = (mH * H1 + mF * F1) / (mH + mF)
com2 = (mH * H2 + mF * F2) / (mH + mF)
c1toc2 = com2 - com1
print(com1,com2,c1toc2)
fh1 = H1 - F1
fh2 = H2 - F2
R = np.sqrt(np.dot(c1toc2,c1toc2))
r1 = np.sqrt(np.dot(fh1,fh1))
r2 = np.sqrt(np.dot(fh2,fh2))
theta1 = (180.0 / np.pi) * np.arccos(np.dot(fh1,c1toc2) / (r1 * R))
theta2 = (180.0 / np.pi) * np.arccos(np.dot(fh2,c1toc2) / (r2 * R))
#phi is either 0 or 180 degrees
print(R,r1,r2,theta1,theta2)
c1tof = np.sqrt(np.dot(com1 - F1,com1 - F1))
c1toh = np.sqrt(np.dot(com1 - H1,com1 - H1))
c2tof = np.sqrt(np.dot(com2 - F2,com2 - F2))
c2toh = np.sqrt(np.dot(com2 - H2,com2 - H2))
print(c1tof,c1toh,c2tof,c2toh)

To determine the asymptotic interaction energy term, we need to find the dipole moment and the quadrupole moment of the HF molecule. Let's calculate those moments at the CCSD level.

For the quadrupole moment, we need to transform the quantity computed by Psi4
$$ Q'_{zz} = \sum_{a} q_a z_a^2 $$
(the sum goes over all particles $a$, electron and nuclei, $q_a$ is the particle charge, and $z_a$ its $z$ coordinate), to the corresponding traceless quadrupole
$$ Q_{zz} = \sum_{a} q_a (\frac{3}{2}z_a^2 -\frac{1}{2}r_a^2)$$

We will ignore the dipole-octupole interactions that vanish with intermolecular distance like $R^{-5}$ just like the quadrupole-quadrupole one.

In [ ]:
#let's calculate the dipole and quadrupole moment for a HF molecule
HF_mol = psi4.geometry("""
  F
  H F 0.921
  units angstrom
  symmetry c1
  """)

psi4.properties('ccsd')
print(psi4.variable('CCSD DIPOLE'))
print(psi4.variable('CCSD QUADRUPOLE'))
dip = psi4.variable('CCSD DIPOLE')[2]

#we need a traceless quadrupole and that's not exactly what we have
trace = (psi4.variable('CCSD QUADRUPOLE')[0][0]+psi4.variable('CCSD QUADRUPOLE')[1][1]+psi4.variable('CCSD QUADRUPOLE')[2][2])/3.0
print("Traceless quadrupole (up to a 3/2 factor???)")
print(psi4.variable('CCSD QUADRUPOLE')[0][0]-trace)
print(psi4.variable('CCSD QUADRUPOLE')[1][1]-trace)
print(psi4.variable('CCSD QUADRUPOLE')[2][2]-trace)
quad = 1.5 * (psi4.variable('CCSD QUADRUPOLE')[2][2]-trace) # probably normalized just like in Stone's book

In [ ]:
#These formulas are for two linear molecules
def dipole_dipole(R,theta1,theta2,phi,mu1,mu2):
# formula source: Stone's book Eq. (3.2.6)
    angular = 2.0 * np.cos(theta1) * np.cos(theta2)
    angular -= np.sin(theta1) * np.sin(theta2) * np.cos(phi)
    return (- mu1 * mu2 / np.power(R,3)) * angular * 627.509

def dipole_quadrupole(R,theta1,theta2,phi,mu1,q2):
# formula source: Stone's book Eq. (3.2.9)
    angular = np.cos(theta1) * (3.0 * np.cos(theta2) * np.cos(theta2) - 1.0)
    angular -= np.sin(theta1) * np.sin(2.0 * theta2) * np.cos(phi)
    angular *= 1.5
    return (mu1 * q2 / np.power(R,4)) * angular * 627.509

def quadrupole_quadrupole(R,theta1,theta2,phi,q1,q2):
# formula source: Stone's book Eq. (3.2.8)
    angular = 1.0
    angular -= 5.0 * np.cos(theta1) * np.cos(theta1)
    angular -= 5.0 * np.cos(theta2) * np.cos(theta2)
    angular -= 15.0 * np.cos(theta1) * np.cos(theta1) * np.cos(theta2) * np.cos(theta2)
    angpart = 4.0 * np.cos(theta1) * np.cos(theta2) - np.sin(theta1) * np.sin(theta2) * np.cos(phi)
    angular += 2.0 * angpart * angpart
    angular *= 0.75
    return (q1 * q2 / np.power(R,5)) * angular * 627.509

We will now compute MP2 and SAPT0 on a grid of points containing the full range of angles $\theta_1$ and $\theta_2$ (the dihedral angle $\phi$ will be kept at 180$^{\circ}$ so that the system remains planar). Then, we compare the MP2 interaction energy and the SAPT0 electrostatic energy with the asymptotic dipole-dipole, dipole-quadrupole, and quadrupole-quadrupole terms.

In [ ]:
#let's use MP2 but calculate the interaction energies on a 3D grid

grid_R = [2.5,2.7164726683628464,3.0,3.5,4.0,5.0,6.0,8.0,10.0,12.0]
grid_t1 = [0.0,30.0,60.0,90.0,120.0,150.0,180.0]
grid_t2 = [0.0,30.0,60.0,90.0,120.0,150.0,180.0]
emp2_grid = np.zeros((len(grid_R),len(grid_t1),len(grid_t2),3))
e_asymp = np.zeros((len(grid_R),len(grid_t1),len(grid_t2),3))
psi4.set_options({'basis': 'aug-cc-pVDZ',
                  'scf_type': 'df',
                  'df_basis_scf': 'aug-cc-pvdz-jkfit',
                  'df_basis_mp2': 'aug-cc-pvdz-ri'})
psi4.core.clean()

for i in range(len(grid_R)):
  for j in range(len(grid_t1)):
    for k in range(len(grid_t2)):
      dimer = psi4.geometry("""
      X1
      X2 X1 """+str(grid_R[i])+"""
      F1 X1 0.04633859816675236 X2 """+str(180.0-grid_t1[j])+"""
      H1 X1 0.8735455679809765 X2 """+str(grid_t1[j])+""" F1 180.0
      --
      F2 X2 0.046477972558587446 X1 """+str(grid_t2[k])+""" F1 180.0
      H2 X2 0.8761729647321557 X1 """+str(180.0-grid_t2[k])+""" F2 180.0
      units angstrom
      symmetry c1
      """)

      psi4.energy('mp2')   
      emp2_grid[i,j,k,0] = psi4.variable('MP2 TOTAL ENERGY')
      psi4.core.clean()

      monomerA = psi4.geometry("""
      X1
      X2 X1 """+str(grid_R[i])+"""
      F1 X1 0.04633859816675236 X2 """+str(180.0-grid_t1[j])+"""
      H1 X1 0.8735455679809765 X2 """+str(grid_t1[j])+""" F1 180.0
      --
      Gh(F2) X2 0.046477972558587446 X1 """+str(grid_t2[k])+""" F1 180.0
      Gh(H2) X2 0.8761729647321557 X1 """+str(180.0-grid_t2[k])+""" F2 180.0
      units angstrom
      symmetry c1
      """)

      psi4.energy('mp2')   
      emp2_grid[i,j,k,1] = psi4.variable('MP2 TOTAL ENERGY')
      psi4.core.clean()

      monomerB = psi4.geometry("""
      X1
      X2 X1 """+str(grid_R[i])+"""
      Gh(F1) X1 0.04633859816675236 X2 """+str(180.0-grid_t1[j])+"""
      Gh(H1) X1 0.8735455679809765 X2 """+str(grid_t1[j])+""" F1 180.0
      --
      F2 X2 0.046477972558587446 X1 """+str(grid_t2[k])+""" F1 180.0
      H2 X2 0.8761729647321557 X1 """+str(180.0-grid_t2[k])+""" F2 180.0
      units angstrom
      symmetry c1
      """)

      psi4.energy('mp2')   
      emp2_grid[i,j,k,2] = psi4.variable('MP2 TOTAL ENERGY')
      psi4.core.clean()
    print('Completed MP2 calculations for angle',j+1)
  print('Completed MP2 calculations for distance',i+1)
    
eintmp2_grid = (emp2_grid[:,:,:,0] - emp2_grid[:,:,:,1] - emp2_grid[:,:,:,2]) * 627.509

print ('MP2 PES',eintmp2_grid)

In [ ]:
for i in range(len(grid_R)):
  for j in range(len(grid_t1)):
    for k in range(len(grid_t2)):
      R = grid_R[i] / 0.529177209
      t1 = grid_t1[j] * (np.pi/180.0)
      t2 = grid_t2[k] * (np.pi/180.0)
      e_asymp[i,j,k,0] = dipole_dipole(R,t1,t2,np.pi,dip,dip)
      e_asymp[i,j,k,1] = dipole_quadrupole(R,t1,t2,np.pi,dip,quad)
      e_asymp[i,j,k,1] += dipole_quadrupole(R,np.pi-t2,np.pi-t1,np.pi,dip,quad)
      e_asymp[i,j,k,2] = quadrupole_quadrupole(R,t1,t2,np.pi,quad,quad)
        
print ('Dipole-dipole PES\n',e_asymp[:,:,:,0])

In [ ]:
plt.close()
fig, ax = plt.subplots(3, 2, figsize=(8,12), height_ratios=[1.0,1.0,1.0])
for i in range(3):
    for j in range(2):
        ax[i][j].set_aspect('equal')
ax[0][0].contourf(grid_t1, grid_t2, eintmp2_grid[0,:,:], cmap="summer")
#ax[0][0].contour(grid_t1, grid_t2, e_asymp[0,:,:,0], cmap="winter")
ax[0][1].contourf(grid_t1, grid_t2, eintmp2_grid[1,:,:], cmap="summer")
ax[1][0].contourf(grid_t1, grid_t2, eintmp2_grid[3,:,:], cmap="summer")
ax[1][1].contourf(grid_t1, grid_t2, eintmp2_grid[5,:,:], cmap="summer")
ax[2][0].contourf(grid_t1, grid_t2, eintmp2_grid[8,:,:], [-0.1,-0.08,-0.06,-0.04,-0.02,0.0,0.02,0.04,0.06,0.08,0.10,0.12,0.14], cmap="summer")
ax[2][0].contour(grid_t1, grid_t2, e_asymp[8,:,:,0], [-0.1,-0.08,-0.06,-0.04,-0.02,0.0,0.02,0.04,0.06,0.08,0.10,0.12,0.14], cmap="winter")
ax[2][0].contour(grid_t1, grid_t2, e_asymp[8,:,:,0]+e_asymp[8,:,:,1]+e_asymp[8,:,:,2], [-0.1,-0.08,-0.06,-0.04,-0.02,0.0,0.02,0.04,0.06,0.08,0.10,0.12,0.14], cmap="Oranges")
ax[2][1].contourf(grid_t1, grid_t2, eintmp2_grid[9,:,:], [-0.1,-0.08,-0.06,-0.04,-0.02,0.0,0.02,0.04,0.06,0.08,0.10,0.12,0.14], cmap="summer")
ax[2][1].contour(grid_t1, grid_t2, e_asymp[9,:,:,0], [-0.1,-0.08,-0.06,-0.04,-0.02,0.0,0.02,0.04,0.06,0.08,0.10,0.12,0.14], cmap="winter")
ax[2][1].contour(grid_t1, grid_t2, e_asymp[9,:,:,0]+e_asymp[9,:,:,1]+e_asymp[9,:,:,2], [-0.1,-0.08,-0.06,-0.04,-0.02,0.0,0.02,0.04,0.06,0.08,0.10,0.12,0.14], cmap="Oranges")

So the dipole-dipole term nicely explains the angular dependence of the potential. Let's now compare the radial dependence.

In [ ]:
jkpairs = [[0,0],[0,3],[0,6],[3,0],[3,3],[3,6],[6,0],[6,3],[6,6]]
plt.close()
fig, ax = plt.subplots(3, 3, figsize=(9,9), height_ratios=[1.0,1.0,1.0])

nst = 0 # start figure from point nr nst

for jkpair in jkpairs:
    j = jkpair[0]
    k = jkpair[1]
    xpos = j // 3
    ypos = k // 3
    t1 = grid_t1[j]
    t2 = grid_t2[k]
    ax[xpos][ypos].plot(grid_R[nst:],eintmp2_grid[nst:,j,k],'g^',linestyle='-',label='MP2')
    ax[xpos][ypos].plot(grid_R[nst:],e_asymp[nst:,j,k,0],'b+',linestyle='-',label='dipole-dipole')
    ax[xpos][ypos].plot(grid_R[nst:],e_asymp[nst:,j,k,0]+e_asymp[nst:,j,k,1],'c',linestyle='-',label='DD + DQ')
    ax[xpos][ypos].plot(grid_R[nst:],e_asymp[nst:,j,k,0]+e_asymp[nst:,j,k,1]+e_asymp[nst:,j,k,2],'cx',linestyle=' ',label='DD + DQ + QQ')
    if j == 0 and k == 0:
        ax[xpos][ypos].legend(loc='lower right')
    ax[xpos][ypos].set_title(r"$\theta_1=$"+str(t1)+r"$^{\circ}$, $\theta_2=$"+str(t2)+r"$^{\circ}$")

plt.show()

In [ ]:
eelst_sapt0 = np.zeros((len(grid_R),len(grid_t1),len(grid_t2)))
eexch_sapt0 = np.zeros((len(grid_R),len(grid_t1),len(grid_t2)))
eind_sapt0 = np.zeros((len(grid_R),len(grid_t1),len(grid_t2)))
edisp_sapt0 = np.zeros((len(grid_R),len(grid_t1),len(grid_t2)))
esapt_sapt0 = np.zeros((len(grid_R),len(grid_t1),len(grid_t2)))
psi4.set_options({'basis': 'jun-cc-pVDZ',
                  'scf_type': 'df',
                  'df_basis_scf': 'jun-cc-pvdz-jkfit',
                  'df_basis_mp2': 'jun-cc-pvdz-ri',
                  'df_basis_sapt': 'jun-cc-pvdz-ri'})

for i in range(len(grid_R)):
  for j in range(len(grid_t1)):
    for k in range(len(grid_t2)):
      dimer = psi4.geometry("""
      X1
      X2 X1 """+str(grid_R[i])+"""
      F1 X1 0.04633859816675236 X2 """+str(180.0-grid_t1[j])+"""
      H1 X1 0.8735455679809765 X2 """+str(grid_t1[j])+""" F1 180.0
      --
      F2 X2 0.046477972558587446 X1 """+str(grid_t2[k])+""" F1 180.0
      H2 X2 0.8761729647321557 X1 """+str(180.0-grid_t2[k])+""" F2 180.0
      units angstrom
      symmetry c1
      """)

      psi4.core.clean()
      psi4.energy('sapt0') 
        
      eelst_sapt0[i,j,k] = psi4.variable('SAPT ELST ENERGY') * 627.509
      eexch_sapt0[i,j,k] = psi4.variable('SAPT EXCH ENERGY') * 627.509
      eind_sapt0[i,j,k] = psi4.variable('SAPT IND ENERGY') * 627.509
      edisp_sapt0[i,j,k] = psi4.variable('SAPT DISP ENERGY') * 627.509
      esapt_sapt0[i,j,k] = psi4.variable('SAPT TOTAL ENERGY') * 627.509
    
    print('Completed SAPT0 calculations for angle',j+1)
  print('Completed SAPT0 calculations for distance',i+1)

In [ ]:
plt.close()
fig, ax = plt.subplots(3, 3, figsize=(9,9), height_ratios=[1.0,1.0,1.0])

nst = 0 # start figure from point nr nst

for jkpair in jkpairs:
    j = jkpair[0]
    k = jkpair[1]
    xpos = j // 3
    ypos = k // 3
    t1 = grid_t1[j]
    t2 = grid_t2[k]
    ax[xpos][ypos].plot(grid_R[nst:],eintmp2_grid[nst:,j,k],'g^',linestyle='-',label='MP2')
    ax[xpos][ypos].plot(grid_R[nst:],eelst_sapt0[nst:,j,k],'ro',linestyle='-',label='SAPT0 electrostatics')
    ax[xpos][ypos].plot(grid_R[nst:],e_asymp[nst:,j,k,0],'b+',linestyle='-',label='dipole-dipole')
    ax[xpos][ypos].plot(grid_R[nst:],e_asymp[nst:,j,k,0]+e_asymp[nst:,j,k,1],'c',linestyle='-',label='DD + DQ')
    ax[xpos][ypos].plot(grid_R[nst:],e_asymp[nst:,j,k,0]+e_asymp[nst:,j,k,1]+e_asymp[nst:,j,k,2],'cx',linestyle=' ',label='DD + DQ + QQ')
    if j == 0 and k == 0:
        ax[xpos][ypos].legend(loc='lower right')
    ax[xpos][ypos].set_title(r"$\theta_1=$"+str(t1)+r"$^{\circ}$, $\theta_2=$"+str(t2)+r"$^{\circ}$")

plt.show()

Finally, let's examine the differences between MP2 and the asymptotic expansion, and between the SAPT0 electrostatics and the asymptotic expansion. What are the sources of these differences?

*(Besides me being sloppy when it comes to theory levels and basis sets...)*

In [ ]:
plt.close()
fig, ax = plt.subplots(3, 3, figsize=(9,9), height_ratios=[1.0,1.0,1.0])

nst = 0 # start figure from point nr nst

for jkpair in jkpairs:
    j = jkpair[0]
    k = jkpair[1]
    xpos = j // 3
    ypos = k // 3
    t1 = grid_t1[j]
    t2 = grid_t2[k]
    ax[xpos][ypos].plot(grid_R[nst:],eintmp2_grid[nst:,j,k]-(e_asymp[nst:,j,k,0]+e_asymp[nst:,j,k,1]+e_asymp[nst:,j,k,2]),'g^',linestyle='-',label='MP2 - (DD+DQ+QQ)')
    ax[xpos][ypos].plot(grid_R[nst:],eelst_sapt0[nst:,j,k]-(e_asymp[nst:,j,k,0]+e_asymp[nst:,j,k,1]+e_asymp[nst:,j,k,2]),'c^',linestyle='-',label=r'$E^{(10)}_{\rm elst}$ - (DD+DQ+QQ)')
    ax[xpos][ypos].hlines(0.0,2.5,12.0,colors='b')
    if j == 0 and k == 0:
        ax[xpos][ypos].legend(loc='lower right')
    ax[xpos][ypos].set_title(r"$\theta_1=$"+str(t1)+r"$^{\circ}$, $\theta_2=$"+str(t2)+r"$^{\circ}$")

plt.show()